In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
from datetime import datetime
from datetime import timedelta
% matplotlib inline
sns.set(style="darkgrid")

In [2]:
from bs4 import BeautifulSoup
import requests
import pprint
import re

In [191]:
times1 = datetime.now()
df_flight = pd.read_csv('flight_data.csv')
times2 = datetime.now()
print('Time spent: '+ str(times2-times1)) # print the time cost

C:\Users\Qingyu\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (48,77,84,85) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Time spent: 0:01:39.445690


## Extract tail numbers, keep the unique values

In [ ]:
tail_num = pd.Series(df['TAIL_NUM'].unique()).dropna()
tail_num_df = pd.DataFrame()
# create a new DataFrame for tail num 
tail_num_df['tail_num'] = tail_num
tail_num = list(tail_num)

In [ ]:
tail_num

### The url is 'https://registry.faa.gov/aircraftinquiry/NNum_Results.aspx?NNumbertxt=NXXXXX'

In [ ]:
mfr_name = []
model = []
year = []
eng_mfr_name = []
eng_model = []
aircraft_type = []

In [ ]:
times1 = datetime.now()
for url in tail_num:
    if isinstance(url, str):
        response = requests.get(
            'https://registry.faa.gov/aircraftinquiry/NNum_Results.aspx?NNumbertxt=' + url
        )
        soup = BeautifulSoup(response.content, "html.parser")
        # determine if the planes has retired    
        if soup.find('span', id='content_lbMfrName') or soup.find('span', id='content_Label7') or soup.find('span', id='content_Label17'):
            # determine if there is a MFR name
            if soup.find('span', id='content_lbMfrName').string != 'None':
                mfr_name.append(soup.find('span', id='content_lbMfrName').string.strip())
            else:
                mfr_name.append(np.nan)
            # determine if there is a model
            if soup.find('span', id='content_Label7').string != 'None':
                model.append(soup.find('span', id='content_Label7').string.strip())
            else:
                model.append(np.nan)
            # determine if there is a MFR year
            if soup.find('span', id='content_Label17').string != 'None':
                year.append(int(soup.find('span', id='content_Label17').string.strip()))
            # use the year in Certificate Issue Date
            elif soup.find('span', id='content_lbCertDate').string != 'None':
                year.append(int(re.search(
                    r'\w+/\w+/(\w+)', soup.find('span', id='content_lbCertDate').string
                ).group(1).strip()))
            else:
                 year.append(np.nan)
            # determine if there is an eng mfr name
            if soup.find('span', id='content_lbEngMfr').string != 'None':
                eng_mfr_name.append(soup.find('span', id='content_lbEngMfr').string.strip())
            else:
                eng_mfr_name.append(np.nan)
            # determine if there is an eng model
            if soup.find('span', id='content_lbEngModel').string != 'None':
                eng_model.append(soup.find('span', id='content_lbEngModel').string.strip())
            else:
                eng_model.append(np.nan)
            # determine if there is a type
            if soup.find('span', id='content_Label11').string != 'None':
                aircraft_type.append(soup.find('span', id='content_Label11').string.strip())
            else:
                aircraft_type.append(np.nan)
        # the plane has retired
        elif soup.find('span', id='content_drptrDeRegAircraft_lbDeRegMfrName_0') or soup.find('span', id='content_drptrDeRegAircraft_lbDeRegModel_0') or soup.find('span', id='content_drptrDeRegAircraft_lbDeRegYearMfr_0'): 
            if soup.find('span', id='content_drptrDeRegAircraft_lbDeRegMfrName_0').string != 'None':
                mfr_name.append(soup.find('span', id='content_drptrDeRegAircraft_lbDeRegMfrName_0').string.strip())
            else:
                mfr_name.append(np.nan)
            # determine if there is a model
            if soup.find('span', id='content_drptrDeRegAircraft_lbDeRegModel_0').string != 'None':
                model.append(soup.find('span', id='content_drptrDeRegAircraft_lbDeRegModel_0').string.strip())
            else:
                model.append(np.nan)
            # determine if there is a MFR year
            if soup.find('span', id='content_drptrDeRegAircraft_lbDeRegYearMfr_0').string != 'None':
                year.append(int(soup.find('span', id='content_drptrDeRegAircraft_lbDeRegYearMfr_0').string.strip()))
            # use the year in Certificate Issue Date
            elif soup.find('span', id='content_drptrDeRegAircraft_lbDeRegCertDate_0').string != 'None':
                year.append(int(re.search(
                    r'\w+/\w+/(\w+)', soup.find('span', id='content_drptrDeRegAircraft_lbDeRegCertDate_0').string
                ).group(1).strip()))
            else:
                 year.append(np.nan)
            # determine if there is an eng mfr name
            if soup.find('span', id='content_drptrDeRegAircraft_lbDREngMfr_0').string != 'None':
                eng_mfr_name.append(soup.find('span', id='content_drptrDeRegAircraft_lbDREngMfr_0').string.strip())
            else:
                eng_mfr_name.append(np.nan)
            # determine if there is an eng model
            if soup.find('span', id='content_drptrDeRegAircraft_lbDREngModel_0').string != 'None':
                eng_model.append(soup.find('span', id='content_drptrDeRegAircraft_lbDREngModel_0').string.strip())
            else:
                eng_model.append(np.nan)
            aircraft_type.append(np.nan)
        else:
            mfr_name.append(np.nan)
            model.append(np.nan)
            year.append(np.nan)
            eng_mfr_name.append(np.nan)
            eng_model.append(np.nan)
            aircraft_type.append(np.nan)
    else:
        mfr_name.append(np.nan)
        model.append(np.nan)
        year.append(np.nan)
        eng_mfr_name.append(np.nan)
        eng_model.append(np.nan)
        aircraft_type.append(np.nan)
tail_num_df['mfr_name'] = mfr_name
tail_num_df['model'] = model
tail_num_df['year'] = year
tail_num_df['eng_mfr_name'] = eng_mfr_name
tail_num_df['eng_model'] = eng_model
tail_num_df['aircraft_type'] = aircraft_type
times2 = datetime.now()
print('Time spent: '+ str(times2-times1)) # print the time cost
tail_num_df.to_csv('./tail_num/tail_num.csv', index = False)

In [ ]:
if soup.find('span', id='content_lbMfrName'):
    pass
else:
    mfr = soup.find('span', id='content_drptrDeRegAircraft_lbDeRegMfrName_0')
mfr

In [ ]:
# s = soup.find('span', id='content_lbCertDate').string
# s.replace('\\','/')
# re.search(r'\w+/\w+/(\w+)', s).group(1)
int(re.search(r'\w+/\w+/(\w+)', soup.find('span', id='content_lbCertDate').string).group(1).strip())

### <td><span class="Results_DataText" id="content_lbMfrName">BOEING                        </span></td>
### <td><span class="Results_DataText" id="content_Label7">737-832             </span></td>
### <td><span class="Results_DataText" id="content_Label17">2001</span></td>
### <td style="width:30%"><span class="Results_DataText" id="content_lbEngMfr">CFM INTL. </span></td>
### <td><span class="Results_DataText" id="content_lbEngModel">CFM56 SERIES </span></td>
### <td><span class="Results_DataText" id="content_lbCertDate">08/29/2002</span></td>
### deresgistred year <td class="boxed-LightCyan"><span class="Results_DataText" id="content_drptrDeRegAircraft_lbDeRegYearMfr_0">1984</span></td>
### deresgistred cert issue date <td class="boxed-LightCyan"><span class="Results_DataText" id="content_drptrDeRegAircraft_lbDeRegCertDate_0">11/04/1996</span></td></tr>

## N442CA 2 deregistered

In [222]:
df_flight.groupby('OP_UNIQUE_CARRIER').describe()

ACTUAL_ELAPSED_TIME                                       \
                                count        mean         std   min    25%   
OP_UNIQUE_CARRIER                                                            
AA                          1783630.0  165.477011   78.959215  33.0  106.0   
AS                           359069.0  185.860445   92.310207  14.0  121.0   
B6                           567638.0  167.290782   84.356950  20.0   99.0   
DL                          1829688.0  141.608341   75.266592  30.0   87.0   
EV                           806098.0   96.326474   33.687131  20.0   71.0   
F9                           195709.0  168.048368   56.658093  46.0  129.0   
HA                           156448.0  102.923189  124.185712  26.0   39.0   
NK                           286839.0  157.304425   58.178362  43.0  116.0   
OO                          1289592.0  100.858481   43.989006  23.0   68.0   
UA                          1115402.0  182.846757   87.110933  29.0  118.0   
VX                           137939.0  214.274897  110.626232  32.0   96.0   
WN                          2589503.0  121.994139   56.960794  30.0   77.0   

                                         AIR_TIME               ...    \
                     50%    75%    max      count        mean   ...     
OP_UNIQUE_CARRIER                                               ...     
AA                 151.0  205.0  642.0  1783630.0  137.585678   ...     
AS                 160.0  245.0  552.0   359069.0  162.605764   ...     
B6                 157.0  195.0  688.0   567638.0  143.055800   ...     
DL                 117.0  175.0  784.0  1829688.0  117.144332   ...     
EV                  92.0  117.0  574.0   806098.0   71.151879   ...     
F9                 156.0  202.0  714.0   195709.0  141.460204   ...     
HA                  46.0   56.0  769.0   156448.0   84.446206   ...     
NK                 151.0  184.0  493.0   286839.0  133.103006   ...     
OO                  92.0  122.0  460.0  1289592.0   74.341257   ...     
UA                 162.0  237.0  778.0  1115402.0  156.656927   ...     
VX                 203.0  319.0  528.0   137939.0  188.568150   ...     
WN                 108.0  152.0  540.0  2589503.0  103.674455   ...     

                  WHEELS_ON               YEAR                                 \
                        75%     max      count         mean       std     min   
OP_UNIQUE_CARRIER                                                               
AA                   1918.0  2400.0  1810843.0  2016.494989  0.499975  2016.0   
AS                   1952.0  2400.0   362348.0  2016.510747  0.499885  2016.0   
B6                   1931.0  2400.0   581127.0  2016.513922  0.499807  2016.0   
DL                   1920.0  2400.0  1846306.0  2016.500220  0.500000  2016.0   
EV                   1833.0  2400.0   830531.0  2016.408824  0.491617  2016.0   
F9                   1943.0  2400.0   198148.0  2016.519950  0.499603  2016.0   
HA                   1822.0  2400.0   156961.0  2016.510777  0.499885  2016.0   
NK                   1941.0  2400.0   295017.0  2016.531556  0.499004  2016.0   
OO                   1845.0  2400.0  1312460.0  2016.538323  0.498529  2016.0   
UA                   1923.0  2359.0  1129548.0  2016.517447  0.499696  2016.0   
VX                   1956.0  2400.0   140102.0  2016.506638  0.499958  2016.0   
WN                   1919.0  2400.0  2628888.0  2016.505706  0.499968  2016.0   

                                                   
                      25%     50%     75%     max  
OP_UNIQUE_CARRIER                                  
AA                 2016.0  2016.0  2017.0  2017.0  
AS                 2016.0  2017.0  2017.0  2017.0  
B6                 2016.0  2017.0  2017.0  2017.0  
DL                 2016.0  2017.0  2017.0  2017.0  
EV                 2016.0  2016.0  2017.0  2017.0  
F9                 2016.0  2017.0  2017.0  2017.0  
HA                 2016.0  2017.0  2017.0  2017.0  
NK             

In [323]:
# check if there is a tail num has 2 or more carrier
tn_carr = df_flight.groupby('TAIL_NUM').OP_CARRIER.unique()
one_to_one = True
one_to_more = []
for i in range(tn_carr.shape[0]):
    if len(tn_carr[i])> 1:
        one_to_one = False
        one_to_more.append(i)
tn_carr[one_to_more]

TAIL_NUM
N603QX    [OO, EV]
N613QX    [OO, EV]
N616QX    [OO, EV]
N617QX    [OO, EV]
N868AS    [EV, OO]
N870AS    [EV, OO]
N871AS    [EV, OO]
N873AS    [EV, OO]
N876AS    [EV, OO]
N877AS    [EV, OO]
N878AS    [EV, OO]
N879AS    [EV, OO]
N880AS    [EV, OO]
N881AS    [EV, OO]
N882AS    [EV, OO]
N884AS    [EV, OO]
N900EV    [EV, OO]
N901EV    [EV, OO]
N902EV    [EV, OO]
N903EV    [EV, OO]
N904EV    [EV, OO]
N905EV    [EV, OO]
N906EV    [EV, OO]
N907EV    [EV, OO]
N908EV    [EV, OO]
N909EV    [EV, OO]
N914EV    [EV, OO]
N916EV    [EV, OO]
N917EV    [EV, OO]
N919EV    [EV, OO]
N920EV    [EV, OO]
N921EV    [EV, OO]
N922EV    [EV, OO]
N923EV    [EV, OO]
N926EV    [EV, OO]
N931EV    [EV, OO]
N933EV    [EV, OO]
N936EV    [EV, OO]
N937EV    [EV, OO]
Name: OP_CARRIER, dtype: object

In [326]:
tn_carr[0]

array(['WN'], dtype=object)

In [284]:
t_num_df = df_flight.groupby('TAIL_NUM').head(1)[['TAIL_NUM', 'OP_CARRIER']].dropna().rename(index=str, columns={'TAIL_NUM': 'tail_num', 'OP_CARRIER': 'carrier'})

In [286]:
t_num_df.to_csv('./tail_num/tail_num_list.csv', index=False)

In [352]:
t_num_df[(t_num_df['tail_num'].str.match(r'^N\w{3}US$')) & (t_num_df['carrier'] == 'AA')].sort_values(by=['tail_num'])

tail_num carrier
21468    N103US      AA
21496    N107US      AA
21410    N111US      AA
21741    N112US      AA
21601    N118US      AA
21560    N119US      AA
22019    N122US      AA
21523    N124US      AA
21337    N163US      AA
21200    N165US      AA
21476    N167US      AA
21467    N170US      AA
21562    N171US      AA
21197    N172US      AA
37805    N173US      AA
21475    N174US      AA
21604    N177US      AA
21585    N178US      AA
21189    N180US      AA
21286    N184US      AA
21172    N186US      AA
21176    N187US      AA
21474    N188US      AA
375057   N704US      AA
370198   N712US      AA
21469    N714US      AA
21265    N722US      AA
21389    N730US      AA
21375    N732US      AA
21638    N737US      AA
21491    N738US      AA
21431    N749US      AA
21236    N752US      AA
21157    N753US      AA
21524    N755US      AA
21839    N756US      AA
21203    N758US      AA
21580    N760US      AA
37512    N762US      AA
21479    N763US      AA
21223    N764US      AA
21220    N765US      AA
21300    N766US      AA
22118    N768US      AA
369784   N769US      AA
21555    N913US      AA
21150    N915US      AA
21433    N916US      AA
21418    N918US      AA
21528    N919US      AA
21381    N920US      AA
21179    N921US      AA
21385    N922US      AA
21229    N923US      AA
21427    N924US      AA

In [ ]:
idx = list(np.array(df.iloc[:, 1:5].dropna(how='all').index))
df = df.iloc[idx]

In [653]:
df_missing = df[df.iloc[:, 1:5].isnull().all(axis=1)]
df_missing = df_missing.dropna(how='all')

In [655]:
scrape('N365AA')

tail_num    mfr_name  model  year cert_year eng_mfr_name eng_model  \
0   N365AA  AGUSTA SPA  A109E  2001      2014  P&W; CANADA    PW206C   

  aircraft_type  
0    Rotorcraft

In [841]:
df = pd.read_csv('./tail_num/new_all_tail_num.csv')
df.groupby('mfr_name').describe()

cert_year                                  \
                                  count         mean        std     min   
mfr_name                                                                  
AIRBUS                            939.0  2012.959531   4.166633  2001.0   
AIRBUS INDUSTRIE                  614.0  2008.043974   6.074552  1998.0   
BOEING                           2687.0  2008.571269   7.018222  1986.0   
BOMBARDIER INC                    426.0  2009.528169   5.903571  2000.0   
CANADAIR                            6.0  2011.000000   3.286335  2008.0   
EMBRAER                           296.0  2009.138514   6.017861  2000.0   
EMBRAER S A                        98.0  2015.581633   1.014569  2014.0   
MCDONNELL DOUGLAS                 144.0  2007.388889   9.722809  1986.0   
MCDONNELL DOUGLAS AIRCRAFT CO     103.0  2003.184466  12.499998  1988.0   
MCDONNELL DOUGLAS CORPORATION      14.0  1992.000000   0.000000  1992.0   
ROBINSON HELICOPTER COMPANY         0.0          NaN        NaN     NaN   
SILVAIRE                            1.0  2002.000000        NaN  2002.0   

                                                                 year  \
                                  25%     50%     75%     max   count   
mfr_name                                                                
AIRBUS                         2010.0  2015.0  2016.0  2018.0   920.0   
AIRBUS INDUSTRIE               2004.0  2009.0  2013.0  2018.0   595.0   
BOEING                         2001.0  2010.0  2015.0  2018.0  2647.0   
BOMBARDIER INC                 2004.0  2009.0  2017.0  2018.0   410.0   
CANADAIR                       2008.0  2011.0  2014.0  2014.0     6.0   
EMBRAER                        2004.0  2007.5  2015.0  2018.0   288.0   
EMBRAER S A                    2015.0  2016.0  2016.0  2017.0    98.0   
MCDONNELL DOUGLAS              1999.0  2011.5  2017.0  2018.0   136.0   
MCDONNELL DOUGLAS AIRCRAFT CO  1990.5  2007.0  2016.0  2018.0   103.0   
MCDONNELL DOUGLAS CORPORATION  1992.0  1992.0  1992.0  1992.0    14.0   
ROBINSON HELICOPTER COMPANY       NaN     NaN     NaN     NaN     0.0   
SILVAIRE                       2002.0  2002.0  2002.0  2002.0     1.0   

                                                                              \
                                      mean       std     min     25%     50%   
mfr_name                                                                       
AIRBUS                         2010.727174  4.871417  2001.0  2006.0  2012.0   
AIRBUS INDUSTRIE               1999.931092  4.718717  1990.0  1998.0  2000.0   
BOEING                         2004.333207  7.794656  1984.0  1999.0  2002.0   
BOMBARDIER INC                 2003.775610  2.688087  1998.0  2002.0  2003.0   
CANADAIR                       1997.000000  0.000000  1997.0  1997.0  1997.0   
EMBRAER                        2004.687500  3.828135  1999.0  2002.0  2004.0   
EMBRAER S A                    2015.581633  1.014569  2014.0  2015.0  2016.0   
MCDONNELL DOUGLAS              1993.235294  4.517022  1986.0  1988.0  1995.0   
MCDONNELL DOUGLAS AIRCRAFT CO  1989.834951  1.771767  1987.0  1988.0  1990.0   
MCDONNELL DOUGLAS CORPORATION  1991.928571  0.267261  1991.0  1992.0  1992.0   
ROBINSON HELICOPTER COMPANY            NaN       NaN     NaN     NaN     NaN   
SILVAIRE                       1959.000000       NaN  1959.0  1959.0  1959.0   

                                               
                                  75%     max  
mfr_name                                       
AIRBUS                         2015.0  2017.0  
AIRBUS INDUSTRIE               2001.0  2014.0  
BOEING                         2012.0  2017.0  
BOMBARDIER INC                 2005.0  2011.0  
CANADAIR                       1997.0  1997.0  
EMBRAER                        2007.0  2017.0  
EMBRAER S A                    2016.0  2017.0  
MCDONNELL DOUGLAS              1997.0  1999.0  
MCDONNELL DOUGLAS AIRCRAFT CO  1991.0  1997.0  
MCDONNELL DOUGLAS CORPORATION  

In [845]:
df[df['tail_num'] == 'N555AA']

tail_num           mfr_name           model    year  cert_year  \
4696   N555AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1991.0     2012.0   

     eng_mfr_name    eng_model            aircraft_type  
4696        P & W  JT8D SERIES  Fixed Wing Multi-Engine

In [647]:
df_missing = df[df.iloc[:, 1:5].isnull().all(axis=1)]
df_missing = df_missing.dropna(how='all')

In [844]:
df[df['mfr_name'] == 'MCDONNELL DOUGLAS']

tail_num           mfr_name           model    year  cert_year  \
15     N905DA  MCDONNELL DOUGLAS        MD-90-30  1995.0     1995.0   
30     N955DN  MCDONNELL DOUGLAS        MD-90-30  1996.0     2011.0   
38     N951DN  MCDONNELL DOUGLAS        MD-90-30  1997.0     2013.0   
64     N906DA  MCDONNELL DOUGLAS        MD-90-30  1995.0     1995.0   
86     N910DN  MCDONNELL DOUGLAS        MD-90-30  1995.0     1996.0   
138    N921DN  MCDONNELL DOUGLAS        MD-90-30     NaN     2010.0   
182    N958DN  MCDONNELL DOUGLAS        MD-90-30  1997.0     2011.0   
184    N954DN  MCDONNELL DOUGLAS        MD-90-30  1996.0     2011.0   
186    N942DN  MCDONNELL DOUGLAS        MD-90-30  1997.0     2012.0   
187    N935DN  MCDONNELL DOUGLAS        MD-90-30  1996.0     2016.0   
212    N913DN  MCDONNELL DOUGLAS        MD-90-30  1996.0     1997.0   
216    N940DN  MCDONNELL DOUGLAS        MD-90-30  1997.0     2012.0   
233    N911DA  MCDONNELL DOUGLAS        MD-90-30  1995.0     1996.0   
247    N956DN  MCDONNELL DOUGLAS        MD-90-30  1997.0     2011.0   
256    N907DA  MCDONNELL DOUGLAS        MD-90-30  1995.0     1995.0   
292    N918DH  MCDONNELL DOUGLAS        MD-90-30  1996.0     2010.0   
296    N943DN  MCDONNELL DOUGLAS        MD-90-30  1997.0     2012.0   
299    N946DN  MCDONNELL DOUGLAS        MD-90-30  1995.0     2012.0   
301    N930DN  MCDONNELL DOUGLAS        MD-90-30  1996.0     2015.0   
303    N957DN  MCDONNELL DOUGLAS        MD-90-30  1997.0     2011.0   
320    N920DN  MCDONNELL DOUGLAS        MD-90-30     NaN     2010.0   
330    N901DA  MCDONNELL DOUGLAS        MD-90-30  1995.0     1995.0   
332    N950DN  MCDONNELL DOUGLAS        MD-90-30     NaN     2013.0   
339    N933DN  MCDONNELL DOUGLAS        MD-90-30  1997.0     2016.0   
345    N934DN  MCDONNELL DOUGLAS        MD-90-30  1996.0     2016.0   
354    N939DN  MCDONNELL DOUGLAS        MD-90-30  1996.0     2012.0   
399    N903DA  MCDONNELL DOUGLAS        MD-90-30  1995.0     1995.0   
416    N938DN  MCDONNELL DOUGLAS        MD-90-30  1995.0     2012.0   
440    N915DN  MCDONNELL DOUGLAS        MD-90-30  1996.0     1997.0   
444    N937DN  MCDONNELL DOUGLAS        MD-90-30     NaN     2012.0   
...       ...                ...             ...     ...        ...   
4691   N496AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1989.0     2016.0   
4692   N4YRAA  MCDONNELL DOUGLAS  DC-9-83(MD-83)  1999.0     1999.0   
4693   N472AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1988.0     2017.0   
4694   N4YFAA  MCDONNELL DOUGLAS  DC-9-83(MD-83)  1999.0     1999.0   
4695   N493AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1989.0     2017.0   
4696   N555AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1991.0     2012.0   
4697   N485AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1988.0     2017.0   
4716   N4YDAA  MCDONNELL DOUGLAS  DC-9-83(MD-83)  1999.0     1999.0   
4717   N513AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1990.0     2013.0   
4719   N4UBAA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1987.0     2018.0   
4720   N425AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1986.0     1987.0   
4721   N4XCAA  MCDONNELL DOUGLAS  DC-9-83(MD-83)  1997.0     2001.0   
4722   N571AA  MCDONNELL DOUGLAS  DC-9-83(MD-83)  1987.0     2017.0   
4782   N469AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1988.0     2017.0   
4783   N4XHAA  MCDONNELL DOUGLAS  DC-9-83(MD-83)  1998.0     2016.0   
4784   N481AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1988.0     2017.0   
4785   N550AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1991.0     2012.0   
4786   N470AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1988.0     2017.0   
4800   N4WTAA  MCDONNELL DOUGLAS  DC-9-83(MD-83)  1992.0     2018.0   
4801   N4YNAA  MCDONNELL DOUGLAS  DC-9-83(MD-83)  1999.0     1999.0   
4802   N424AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1986.0     1986.0   
4803   N488AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1988.0     1989.0   
4804   N4XYAA  MCDONNELL DOUGLAS  DC-9-83(MD-83)  1999.0     1999.0   
4833   N528AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1990.0     1990.0   
4834   N4WMAA  MCDONNELL DOUG

In [ ]:
update_result_by_tail_num(df, 'N356AA', 'N39356')

In [656]:
tail_num_convert[tail_num_convert['old_tail_num'] == 'N528JB']

Empty DataFrame
Columns: [old_tail_num, new_tail_num]
Index: []

In [599]:
rescrape(df, 'N403AA')

In [580]:
scrape('N902FR')

tail_num          mfr_name     model  year cert_year eng_mfr_name eng_model  \
0   N902FR  AIRBUS INDUSTRIE  A319-112  2001      2002    CFM INTL.     13802   

  aircraft_type  
0           NaN

In [702]:
df_flight[df_flight['TAIL_NUM'] == 'N187SW'].loc[:, 'DISTANCE']

5797167    748.0
5797211    748.0
Name: DISTANCE, dtype: float64

In [887]:
df_flight[df_flight['TAIL_NUM'] == 'N492SW'].iloc[:, :20]
# df_flight[(df_flight['TAIL_NUM'] == 'N187SW') & (df_flight['MONTH'] >= 10)].iloc[:, :20]

YEAR  QUARTER  MONTH  DAY_OF_MONTH  DAY_OF_WEEK     FL_DATE  \
133551    2016        1      1            25            1  2016-01-25   
133552    2016        1      1            31            7  2016-01-31   
133564    2016        1      1             4            1  2016-01-04   
133579    2016        1      1             4            1  2016-01-04   
133582    2016        1      1             7            4  2016-01-07   
133586    2016        1      1            20            3  2016-01-20   
133589    2016        1      1            13            3  2016-01-13   
133594    2016        1      1            13            3  2016-01-13   
133597    2016        1      1             1            5  2016-01-01   
133598    2016        1      1             4            1  2016-01-04   
133606    2016        1      1             7            4  2016-01-07   
133621    2016        1      1            30            6  2016-01-30   
133622    2016        1      1            29            5  2016-01-29   
133628    2016        1      1            10            7  2016-01-10   
133646    2016        1      1            30            6  2016-01-30   
133647    2016        1      1            10            7  2016-01-10   
133665    2016        1      1            29            5  2016-01-29   
133676    2016        1      1             7            4  2016-01-07   
133680    2016        1      1            13            3  2016-01-13   
133686    2016        1      1            20            3  2016-01-20   
134626    2016        1      1            20            3  2016-01-20   
134633    2016        1      1             1            5  2016-01-01   
134656    2016        1      1            31            7  2016-01-31   
134664    2016        1      1            31            7  2016-01-31   
134680    2016        1      1             1            5  2016-01-01   
134708    2016        1      1             9            6  2016-01-09   
134711    2016        1      1            26            2  2016-01-26   
134717    2016        1      1             9            6  2016-01-09   
134725    2016        1      1            26            2  2016-01-26   
134752    2016        1      1             2            6  2016-01-02   
...        ...      ...    ...           ...          ...         ...   
11105607  2017        4     10            14            6  2017-10-14   
11105608  2017        4     10            14            6  2017-10-14   
11105744  2017        4     10            14            6  2017-10-14   
11105770  2017        4     10            14            6  2017-10-14   
11107273  2017        4     10            15            7  2017-10-15   
11107274  2017        4     10            15            7  2017-10-15   
11111442  2017        4     10            28            6  2017-10-28   
11111462  2017        4     10            28            6  2017-10-28   
11111463  2017        4     10            28            6  2017-10-28   
11111588  2017        4     10            28            6  2017-10-28   
11111589  2017        4     10            28            6  2017-10-28   
11113043  2017        4     10            29            7  2017-10-29   
11113044  2017        4     10            29            7  2017-10-29   
11113047  2017        4     10            29            7  2017-10-29   
11113054  2017        4     10            29            7  2017-10-29   
11113055  2017        4     10            29            7  2017-10-29   
11113065  2017        4     10            29            7  2017-10-29   
11113087  2017        4     10            29            7  2017-10-29   
11113088  2017        4     10            29            7  2017-10-29   
11115044  2017        4     10            30            1  2017-10-30   
11115045  2017        4     10            30            1  2017-10-30   
11115063  2017        4     10            30            1  2017-10-30   
11115089  2017        4     10            30            1  2017-10-3

In [891]:
df = pd.read_csv('./tail_num/new_all_tail_num.csv')
df_missing = df[df.iloc[:, 1:5].isnull().all(axis=1)]
df_missing

tail_num mfr_name model  year  cert_year eng_mfr_name eng_model  \
4042   PLANET      NaN   NaN   NaN        NaN          NaN       NaN   
4062   N187SW      NaN   NaN   NaN        NaN          NaN       NaN   
4281   N101NK      NaN   NaN   NaN        NaN          NaN       NaN   

     aircraft_type  
4042           NaN  
4062           NaN  
4281           NaN

In [833]:
df = update_result_by_input_info(df)
df.to_csv('./tail_num/new_all_tail_num.csv', index=False)

Input the tail number:
N365AA
Current info for this tail number:

  tail_num mfr_name model  year  cert_year eng_mfr_name eng_model  \
0   N365AA      NaN   NaN   NaN        NaN          NaN       NaN   

  aircraft_type  
0           NaN   

Do you want to update mfr_name ? Y/n
Y
Input a new mfr_name or Unknown:
BOEING
Do you want to update model ? Y/n
Y
Input a new model or Unknown:
767-323ER
Do you want to update year ? Y/n
Y
Input a new year or Unknown:
1988
Do you want to update cert_year ? Y/n
Y
Input a new cert_year or Unknown:
1988
Do you want to update eng_mfr_name ? Y/n
Y
Input a new eng_mfr_name or Unknown:
GE
Do you want to update eng_model ? Y/n
Y
Input a new eng_model or Unknown:
CF6-80C2B6
Do you want to update aircraft_type ? Y/n
Y
Input a new aircraft_type or Unknown:
Fixed Wing Multi-Engine
  tail_num mfr_name      model  year cert_year eng_mfr_name   eng_model  \
0   N365AA   BOEING  767-323ER  1988      1988           GE  CF6-80C2B6   

             aircraft_type  


In [ ]:
df.to_csv('./tail_num/new_all_tail_num.csv', index=False)

In [529]:
scrape('N545AA')

tail_num mfr_name       model  year cert_year eng_mfr_name    eng_model  \
0   N545AA    PIPER  PA-32R-300  1976      2013     LYCOMING  TI0-540 SER   

              aircraft_type  
0  Fixed Wing Single-Engine

In [902]:
rescrape(df, 'N492SW')

In [ ]:
df_conv_tail_num = df_conv_tail_num.reset_index(drop=True)

In [ ]:
aa_df = pd.read_csv('./tail_num/aa_current_fleet_tail_num_convert.csv')

In [ ]:
# match AA current fleet
for i in range(df_conv_tail_num.shape[0]):
    for j in range(aa_df.shape[0]):
        if df_conv_tail_num.loc[i, 'old_tail_num'] == aa_df.loc[j, 'old_tail_num']:
            df_conv_tail_num.loc[i, 'new_tail_num'] = aa_df.loc[j, 'new_tail_num']
            break

In [ ]:
aa_ret_df = pd.read_csv('./tail_num/aa_retired_fleet_tail_num_convert.csv')

In [ ]:
# match AA retire fleet
for i in range(df_conv_tail_num.shape[0]):
    if not isinstance(df_conv_tail_num.loc[i, 'new_tail_num'], str):
        for j in range(aa_ret_df.shape[0]):
            if df_conv_tail_num.loc[i, 'old_tail_num'] == aa_ret_df.loc[j, 'old_tail_num']:
                df_conv_tail_num.loc[i, 'new_tail_num'] = aa_ret_df.loc[j, 'new_tail_num']
                break

In [ ]:
# match AA current fleet ignore US/AA in the end
for i in range(df_conv_tail_num.shape[0]):
    if not isinstance(df_conv_tail_num.loc[i, 'new_tail_num'], str):
        for j in range(aa_df.shape[0]):
            if df_conv_tail_num.loc[i, 'old_tail_num'][:-2] == aa_df.loc[j, 'old_tail_num'][:-2]:
                df_conv_tail_num.loc[i, 'new_tail_num'] = aa_df.loc[j, 'new_tail_num']
                break

In [ ]:
df_conv_tail_num.to_csv('./tail_num/tail_num_convert.csv', index = False)

In [ ]:
# df_flight[(df_flight['TAIL_NUM'] == 'N498SW') & (df_flight['MONTH'] > 10)].iloc[:,:11]
df_flight[df_flight['TAIL_NUM'] == 'N498SW'].iloc[:,:11]

# scrape data for matched tail numbers

In [ ]:
mfr_name = []
model = []
year = []
eng_mfr_name = []
eng_model = []
aircraft_type = []

In [ ]:
tail_num = pd.read_csv('./tail_num/tail_num_convert.csv')
tail_num_lst = tail_num['new_tail_num'].tolist()

In [ ]:
tail_num_df = pd.DataFrame()
tail_num_df['tail_num'] = tail_num['old_tail_num']

In [ ]:
tail_num_df_1 = tail_num_df

In [ ]:
tail_num_df = pd.read_csv('./tail_num/matched_tail_num.csv')

In [ ]:
idx = tail_num_df_1.iloc[:, 1:5].dropna(how='all').index.tolist()

In [ ]:
tail_num_df_1 = tail_num_df_1.loc[idx]

In [ ]:
tail_num_df_1 = pd.concat([tail_num_df_1, tail_num_df])

In [ ]:
tail_num_df_1.reset_index(drop=True)
tail_num_df_1.to_csv('./tail_num/all_tail_num.csv', index=False)

In [ ]:
df = pd.read_csv('flight_data.csv')
tail_num = pd.Series(df['TAIL_NUM'].unique()).dropna()
tail_num_df = pd.DataFrame()
# create a new DataFrame for tail num
tail_num_df['tail_num'] = tail_num

In [ ]:
tail_num_df.to_csv('./tail_num/tail_num_list.csv', index=False)

In [ ]:
df_not_aa = tail_num_df[tail_num_df['tail_num'].str.match(r'^N\w{3}AA$').apply(lambda x: not x)]
df_aa = tail_num_df[tail_num_df['tail_num'].str.match(r'^N\w{3}AA$')]
df_tail_num_convert = pd.read_csv('./tail_num/tail_num_convert.csv')

In [ ]:
df_aa = df_aa.rename(index=str, columns={"tail_num": "old_tail_num"})

In [ ]:
df_aa['new_tail_num'] = np.nan
df_aa = df_aa.reset_index(drop=True)

In [ ]:
aa_current = pd.read_csv('./tail_num/aa_current_fleet_tail_num_convert.csv')
# match AA current fleet
for i in range(df_aa.shape[0]):
    for j in range(aa_current.shape[0]):
        if df_aa.loc[i, 'old_tail_num'] \
                == aa_current.loc[j, 'old_tail_num']:
            df_aa.loc[i, 'new_tail_num'] \
                = aa_current.loc[j, 'new_tail_num']
            break
aa_ret = pd.read_csv('./tail_num/aa_retired_fleet_tail_num_convert.csv')
# match AA retire fleet
for i in range(df_aa.shape[0]):
    if not isinstance(df_aa.loc[i, 'new_tail_num'], str):
        for j in range(aa_ret.shape[0]):
            if df_aa.loc[i, 'old_tail_num'] \
                    == aa_ret.loc[j, 'old_tail_num']:
                df_aa.loc[i, 'new_tail_num'] \
                    = aa_ret.loc[j, 'new_tail_num']
                break
# match AA current fleet ignore US/AA in the end
for i in range(df_aa.shape[0]):
    if not isinstance(df_aa.loc[i, 'new_tail_num'], str):
        for j in range(aa_current.shape[0]):
            if df_aa.loc[i, 'old_tail_num'][:-2] \
                    == aa_current.loc[j, 'old_tail_num'][:-2]:
                df_aa.loc[i, 'new_tail_num'] \
                    = aa_current.loc[j, 'new_tail_num']
                break

In [ ]:
df_aa = df_aa.dropna(how='any')

In [ ]:
df_tail_num_convert = pd.concat([df_tail_num_convert, df_aa])

In [ ]:
df_tail_num_convert = df_tail_num_convert.drop_duplicates().reset_index(drop=True)

In [ ]:
df_tail_num_convert.to_csv('./tail_num/tail_num_convert.csv', index=False)

In [ ]:
df_tail_num_convert['old_tail_num']

# Find if it's in convert list then scrape

In [643]:
tail_num_df = pd.read_csv('./tail_num/tail_num_list.csv')
tail_num_convert = pd.read_csv('./tail_num/updated_tail_num_convert.csv')

In [644]:
def find_match(tail_num, df):
    for i in range(df.shape[0]):
        if df.loc[i, 'old_tail_num'] == tail_num:
            return df.loc[i, 'new_tail_num']
    return None

    

In [ ]:
isinstance(find_match('N4YPAA', tail_num_convert), str)

In [645]:
tail_num_list = tail_num_df['tail_num'].tolist()
df = pd.DataFrame()
for tail_num in tail_num_list:
    if isinstance(find_match(tail_num, tail_num_convert), str):  # tail num has to be converted
        result = scrape(find_match(tail_num, tail_num_convert))
        result.loc[0, 'tail_num'] = tail_num
        df = pd.concat([df, result], ignore_index=True)
    else:
        df = pd.concat([df, scrape(tail_num)], ignore_index=True)
df.to_csv('./tail_num/new_all_tail_num.csv', index=False)

In [39]:
df

tail_num          mfr_name     model  year cert_year eng_mfr_name  \
0    N3ALAA            BOEING   737-823  1999      1999    CFM INTL.   
1    N3EFAA            BOEING   737-823  2009      2009     CFM INTL   
2    N3EPAA            BOEING   737-823  2009      2009     CFM INTL   
3    N3JEAA            BOEING   737-823  2012      2012     CFM INTL   
4    N3NDAA            BOEING   737-823  2016      2016     CFM INTL   
5    N3HCAA            BOEING   737-823  2011      2011     CFM INTL   
6    N3ASAA            BOEING   737-823  1999      1999    CFM INTL.   
7    N3HTAA            BOEING   737-823  2012      2012     CFM INTL   
8    N3EJAA            BOEING   737-823  2009      2009     CFM INTL   
9    N3HEAA            BOEING   737-823  2011      2011     CFM INTL   
10   N3KHAA            BOEING   737-823  2013      2013     CFM INTL   
11   N3KNAA            BOEING   737-823  2013      2013     CFM INTL   
12   N3DNAA            BOEING   737-823  2009      2009     CFM INTL   
13   N3GAAA            BOEING   737-823  2010      2010     CFM INTL   
14   N3CCAA            BOEING   737-823  2000      2000    CFM INTL.   
15   N3AHAA            BOEING   737-823  1999      1999    CFM INTL.   
16   N3DAAA            BOEING   737-823  2001      2001    CFM INTL.   
17   N3GJAA            BOEING   737-823  2010      2010     CFM INTL   
18   N3ANAA            BOEING   737-823  1999      1999    CFM INTL.   
19   N3KBAA            BOEING   737-823  2013      2013     CFM INTL   
20   N3FNAA            BOEING   737-823  2010      2010     CFM INTL   
21   N850AA  AIRBUS INDUSTRIE  A321-231  2014      2014          IAE   
22   N855AA            AIRBUS  A321-231  2014      2014          IAE   
23   N864AA            AIRBUS  A321-231  2015      2015          IAE   
24   N862AA            AIRBUS  A321-231  2014      2014          IAE   
25   N859AA            AIRBUS  A321-231  2014      2014          IAE   
26   N865AA            AIRBUS  A321-231  2015      2015          IAE   
27   N861AA            AIRBUS  A321-231  2014      2014          IAE   
28   N857AA            AIRBUS  A321-231  2014      2014          IAE   
29   N863AA            AIRBUS  A321-231  2015      2015          IAE   
30   N852AA  AIRBUS INDUSTRIE  A321-231  2014      2014          IAE   
31   N858AA            AIRBUS  A321-231  2014      2014          IAE   
32   N860AA            AIRBUS  A321-231  2014      2014          IAE   
33   N853AA            AIRBUS  A321-231  2014      2014          IAE   
34   N854AA            AIRBUS  A321-231  2014      2014          IAE   
35   N856AA            AIRBUS  A321-231  2014      2014          IAE   
36   N383AA            BOEING   767-323  1993      2018           GE   
37   N365AA               NaN       NaN   NaN       NaN          NaN   
38   N5ERAA            BOEING   757-223  1999      1999   ROLLS-ROYC   
39   N5ETAA            BOEING   757-223  2001      2001   ROLLS-ROYC   
40   N851AA            AIRBUS  A321-231  2014      2014          IAE   
41   N5FMAA            BOEING   757-223  2002      2002   ROLLS-ROYC   
42   N5EWAA            BOEING   757-223  2001      2001   ROLLS-ROYC   
43   N5DKAA            BOEING   757-223  1993      1993   ROLLS-ROYC   
44   N3AYAA            BOEING   737-823  1999      1999    CFM INTL.   
45   N5FEAA            BOEING   757-223  2001      2001   ROLLS-ROYC   
46   N7ABAA            BOEING   777-223  1999      1999   ROLLS-ROYC   
47   N387AA               NaN       NaN   NaN       NaN          NaN   
48   N346AA            BOEING   767-323  2003      2003           GE   
49   N368AA            BOEING   767-323  1991      2017           GE   

        eng_model            aircraft_type  
0    CFM56 SERIES  Fixed Wing Multi-Engine  
1    CFM56-7B24/3  Fixed Wing Multi-Engine  
2    CFM56-7B24/3  Fixed Wing Multi-Engine  
3     CFM56-7B24E  Fixed Wing Multi-Engine  
4     CFM56-7B24E  Fixed Wing Multi-Engine  
5     CFM56-7B24E  Fixed Wing Multi-Engine  
6    CFM56 SERI

In [11]:
tail_num_list[3836]

'N368AA'

In [15]:
scrape(find_match('N368AA', tail_num_convert))

tail_num mfr_name model year cert_year eng_mfr_name eng_model aircraft_type
0   N368AA      NaN   NaN  NaN       NaN          NaN       NaN           NaN

# Update convert list

In [22]:
tail_num_convert[tail_num_convert['old_tail_num'] == 'N368AA']

old_tail_num new_tail_num
192       N368AA       N396CM

In [153]:
def update_convert_list(tail_num_convert, old_tail_num, new_tail_num):
    match = False
    for i in range(tail_num_convert.shape[0]):
        if tail_num_convert.loc[i, 'old_tail_num'] == old_tail_num:
            match = True
            tail_num_convert.loc[i, 'new_tail_num'] = new_tail_num
            break
    if not match:
        tail_num_convert.loc[tail_num_convert.shape[0], 'old_tail_num'] = old_tail_num
        tail_num_convert.loc[tail_num_convert.shape[0]-1, 'new_tail_num'] = new_tail_num
    return tail_num_convert

In [21]:
update_convert_list(tail_num_convert, 'N368AA', 'N396CM')

old_tail_num new_tail_num
0          N526JB       N526JL
1          N527JB       N527JL
2          N507JB       N507JT
3          N943JB       N943JT
4          N531JB       N531JL
5          N760AS       N807TJ
6          N769AS       N808TJ
7          N767AS       N806TJ
8          N855AS       N987SW
9          N128UA       5N-ADM
10         N608SW       N608SW
11         N617SW       N137CG
12         N606SW       N926XA
13         N4YBAA       N968TW
14         N541AA       N7541A
15         N4YPAA       N980TW
16         N4XBAA       N9615W
17         N4XPAA       N9628W
18         N4UCAA        N955U
19         N4XVAA       N963TW
20         N4XGAA       N9620D
21         N4YEAA       N971TW
22         N4WJAA       N9401W
23         N4YLAA       N9677W
24         N4XWAA       N964TW
25         N3HMAA       N879NN
26         N3BDAA       N927AN
27         N3KJAA       N923NN
28         N3FWAA       N843NN
29         N3ESAA       N816NN
...           ...          ...
1016       N658AA       N658AW
1017       N105AA       N105UW
1018       N207AA       N207UW
1019       N272AA       N272AY
1020       N288AA       N288AY
1021       N277AA       N277AY
1022       N275AA       N275AY
1023       N271AA       N271AY
1024       N270AA       N270AY
1025       N996AA       N996AN
1026       N998AA       N998AN
1027       N929AA       N929AA
1028       N997AA       N997AA
1029       N931AA       N931AM
1030       N905AA       N905AU
1031       N934AA       N934AA
1032       N901AA       N901AA
1033       N930AA       N930AU
1034       N903AA       N903AA
1035       N907AA       N907AA
1036       N928AA       N928AM
1037       N900AA       N900UW
1038       N909AA       N909AM
1039       N933AA       N933AM
1040       N906AA       N906AA
1041       N910AA       N910AU
1042       N932AA       N932AM
1043       N904AA       N904AA
1044       N902AA       N902AA
1045       N376AA       N376AN

[1046 rows x 2 columns]

In [ ]:
tail_num_convert

# Update above

# Update result df by tail num below 

In [532]:
def update_result_by_tail_num(result_df, old_tail_num, new_tail_num):
    # update tail_num_convert dataframe first
    tail_num_convert = pd.read_csv('./tail_num/updated_tail_num_convert.csv')
    tail_num_convert = update_convert_list(tail_num_convert, old_tail_num, new_tail_num)
    tail_num_convert.to_csv('./tail_num/updated_tail_num_convert', index=False)
    # delete the row of old_tail_num in result_df
    result_df = result_df.drop(result_df[result_df['tail_num'] == old_tail_num].index.values).reset_index(drop=True)
    assert isinstance(find_match(old_tail_num, tail_num_convert), str)
    # add new scraping result to result_df
    result = scrape(new_tail_num)
    result.loc[0, 'tail_num'] = old_tail_num
    result_df = pd.concat([result_df, result], ignore_index=True)
    result_df.to_csv('./tail_num/new_all_tail_num.csv', index=False)
    return result_df

In [812]:
def rescrape(result_df, tail_num):
    tail_num_convert = pd.read_csv('./tail_num/updated_tail_num_convert.csv')
    # delete the row of old_tail_num in result_df
    result_df = result_df.drop(result_df[result_df['tail_num'] == tail_num].index.values).reset_index(drop=True)
    if isinstance(find_match(tail_num, tail_num_convert), str):  # tail num has to be converted
        result = scrape(find_match(tail_num, tail_num_convert))
        result.loc[0, 'tail_num'] = tail_num
    else:
        result = scrape(tail_num)
    result_df = pd.concat([result_df, result], ignore_index=True)
    result_df.to_csv('./tail_num/new_all_tail_num.csv', index=False)

In [38]:
df = update_result_by_tail_num(df, 'N368AA', 'N396CM')
df.to_csv('./tail_num/new_all_tail_num.csv', index=False)

In [506]:
tail_num_convert[1040:]

old_tail_num new_tail_num
1040       N906AA       N906AA
1041       N910AA       N910AU
1042       N932AA       N932AM
1043       N904AA       N904AA
1044       N902AA       N902AA
1045       N376AA       N376AN
1046       N966JB       N966JT
1047       N968JB       N968JT
1048       N967JB       N967JT
1049       N985JB       N985JT
1050       N988JB       N988JT
1051       N944JB       N944JT
1052       N981JB       N981JT
1053       N945JB       N945JT
1054       N987JB       N987JT
1055       N989JB       N989JT
1056       N492SW       N492WN
1057       N416SW       N416WN
1058       N949JB       N949JT
1059       N12921        N67VA
1060       N555AA       N555AN
1061       N962JB       N962JT
1062       N983JB       N983JT
1063       N990JB       N990JL
1064       N59034       N2534U
1065       N508JB       N508JL
1066       N969JB       N969JT
1067       N972JB       N972JT
1068       N976JB       N976JT
1069       N950JB       N950JT
1070       N961JB       N961JT
1071       N935AA       N935UW
1072       N980JB       N980JT
1073       N16918        N70VA
1074       N946JB       N946JL
1075       N550AA        N7550
1076       N936AA       N936UW
1077       N528AA       N7528A
1078       N13914        N87VA
1079       N971JB       N971JT
1080       N979JB       N979JT
1081       N425AA       N70425
1082       N965JB       N965JT
1083       N878DA       N878DN
1084       N974JB       N974JT
1085       N537JB       N537JT
1086       N548AA       N7548A
1087       N942AA       N942UW
1088       N941AA       N941UW
1089       N940AA       N940UW
1090       N975JB       N975JT
1091       N547AA       N7547A
1092       N964JB       N964JT

In [32]:
tail_num_convert[tail_num_convert['old_tail_num'] == 'N904AA'].index.values

array([1043], dtype=int64)

In [34]:
tail_num_convert.drop(tail_num_convert[tail_num_convert['old_tail_num'] == 'N904AA'].index.values).reset_index(drop=True)

old_tail_num new_tail_num
0          N526JB       N526JL
1          N527JB       N527JL
2          N507JB       N507JT
3          N943JB       N943JT
4          N531JB       N531JL
5          N760AS       N807TJ
6          N769AS       N808TJ
7          N767AS       N806TJ
8          N855AS       N987SW
9          N128UA       5N-ADM
10         N608SW       N608SW
11         N617SW       N137CG
12         N606SW       N926XA
13         N4YBAA       N968TW
14         N541AA       N7541A
15         N4YPAA       N980TW
16         N4XBAA       N9615W
17         N4XPAA       N9628W
18         N4UCAA        N955U
19         N4XVAA       N963TW
20         N4XGAA       N9620D
21         N4YEAA       N971TW
22         N4WJAA       N9401W
23         N4YLAA       N9677W
24         N4XWAA       N964TW
25         N3HMAA       N879NN
26         N3BDAA       N927AN
27         N3KJAA       N923NN
28         N3FWAA       N843NN
29         N3ESAA       N816NN
...           ...          ...
1015       N995AA       N995AN
1016       N658AA       N658AW
1017       N105AA       N105UW
1018       N207AA       N207UW
1019       N272AA       N272AY
1020       N288AA       N288AY
1021       N277AA       N277AY
1022       N275AA       N275AY
1023       N271AA       N271AY
1024       N270AA       N270AY
1025       N996AA       N996AN
1026       N998AA       N998AN
1027       N929AA       N929AA
1028       N997AA       N997AA
1029       N931AA       N931AM
1030       N905AA       N905AU
1031       N934AA       N934AA
1032       N901AA       N901AA
1033       N930AA       N930AU
1034       N903AA       N903AA
1035       N907AA       N907AA
1036       N928AA       N928AM
1037       N900AA       N900UW
1038       N909AA       N909AM
1039       N933AA       N933AM
1040       N906AA       N906AA
1041       N910AA       N910AU
1042       N932AA       N932AM
1043       N902AA       N902AA
1044       N376AA       N376AN

[1045 rows x 2 columns]

# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
# Update result df by tail num above

In [7]:
df = pd.read_csv('./tail_num/new_all_tail_num.csv')

In [10]:
df.groupby('mfr_name').describe()

cert_year                                  \
                                  count         mean        std     min   
mfr_name                                                                  
AERONCA                             1.0  1948.000000        NaN  1948.0   
AIR TRACTOR INC                     4.0  2016.250000   1.707825  2014.0   
AIRBUS                            879.0  2012.819113   4.225187  2002.0   
AIRBUS INDUSTRIE                  604.0  2007.930464   6.057639  1998.0   
BARD JOHN A                         1.0  2004.000000        NaN  2004.0   
BEER JOHN EDWARD                    0.0          NaN        NaN     NaN   
BELTZ JOE M                         1.0  1981.000000        NaN  1981.0   
BENHAM JOHN                         1.0  2012.000000        NaN  2012.0   
BIGHAM JACK L                       2.0  2011.500000   0.707107  2011.0   
BOEING                           2668.0  2008.540480   7.000044  1986.0   
BOMBARDIER INC                    425.0  2009.510588   5.899353  2000.0   
CANADAIR                            8.0  2011.125000   3.681518  2007.0   
CESSNA                              8.0  1998.250000  22.895726  1949.0   
CIRRUS DESIGN CORP                  4.0  2012.750000   2.872281  2011.0   
DASSAULT                            1.0  2005.000000        NaN  2005.0   
DASSAULT/SUD                        1.0  1985.000000        NaN  1985.0   
DIAMOND AIRCRAFT IND GMBH           1.0  2013.000000        NaN  2013.0   
DIAMOND AIRCRAFT IND INC            1.0  2004.000000        NaN  2004.0   
DJI                                 2.0  2017.500000   0.707107  2017.0   
EMBRAER                           293.0  2008.955631   6.114328  1987.0   
EMBRAER S A                        98.0  2015.581633   1.014569  2014.0   
FLEETWINGS INC                      1.0  1941.000000        NaN  1941.0   
GATES LEAR JET                      1.0  1974.000000        NaN  1974.0   
GULFSTREAM AEROSPACE                2.0  2016.000000   2.828427  2014.0   
ISRAEL AIRCRAFT INDUSTRIES          0.0          NaN        NaN     NaN   
LAMBERT RICHARD                     1.0  2009.000000        NaN  2009.0   
MCDONNELL DOUGLAS                 170.0  2009.035294   9.476782  1986.0   
MCDONNELL DOUGLAS AIRCRAFT CO     103.0  2003.184466  12.499998  1988.0   
MCDONNELL DOUGLAS CORPORATION      14.0  1992.000000   0.000000  1992.0   
MONOCOUPE                           1.0  1942.000000        NaN  1942.0   
MOONEY AIRCRAFT CORP.               1.0  2008.000000        NaN  2008.0   
NAKAMURA HISASHI                    0.0          NaN        NaN     NaN   
PILATUS AIRCRAFT LTD                1.0  2018.000000        NaN  2018.0   
PIPER                               3.0  2012.000000   6.082763  2005.0   
QUEST AIRCRAFT COMPANY LLC          1.0  2018.000000        NaN  2018.0   
ROBINSON HELICOPTER CO              1.0  2013.000000        NaN  2013.0   
ROBINSON HELICOPTER COMPANY         0.0          NaN        NaN     NaN   
SEELBACK JAN/HERB                   0.0          NaN        NaN     NaN   
SIKORSKY                            3.0  2017.333333   1.154701  2016.0   
SILVAIRE                            1.0  2002.000000        NaN  2002.0   
SOCATA                              1.0  2016.000000        NaN  2016.0   
STINNETT RON                        1.0  1987.000000        NaN  1987.0   
TREMBLE JASON M                     1.0  2012.000000        NaN  2012.0   

                                                                   year  \
                                   25%     50%      75%     max   count   
mfr_name                                                                  
AERONCA                        1948.00  1948.0  1948.00  1948.0     1.0   
AIR TRACTOR INC                2015.50  2016.5  2017.25  2018.0     4.0   
AIRBUS                         2009.00  2015.0  2016.00  2018.0   860.0   
AIRBUS INDUSTRIE               2003.75  2009.0  2013.00  2018.0   585.0   
BARD JOHN A                    2004.00  2004.0  2004.0

In [8]:
df_missing = df[df.iloc[:, 1:].isnull().all(axis=1)]

In [9]:
df_missing

tail_num mfr_name model  year  cert_year eng_mfr_name eng_model  \
2733   N128UA      NaN   NaN   NaN        NaN          NaN       NaN   
2919   N608SW      NaN   NaN   NaN        NaN          NaN       NaN   
3836   N368AA      NaN   NaN   NaN        NaN          NaN       NaN   
3838   N365AA      NaN   NaN   NaN        NaN          NaN       NaN   
3848   N387AA      NaN   NaN   NaN        NaN          NaN       NaN   
3854   N377AA      NaN   NaN   NaN        NaN          NaN       NaN   
4077   N382AA      NaN   NaN   NaN        NaN          NaN       NaN   
4087   N378AA      NaN   NaN   NaN        NaN          NaN       NaN   
4282   N498SW      NaN   NaN   NaN        NaN          NaN       NaN   
4321   N886AA      NaN   NaN   NaN        NaN          NaN       NaN   
5030   PLANET      NaN   NaN   NaN        NaN          NaN       NaN   

     aircraft_type  
2733           NaN  
2919           NaN  
3836           NaN  
3838           NaN  
3848           NaN  
3854           NaN  
4077           NaN  
4087           NaN  
4282           NaN  
4321           NaN  
5030           NaN

# Update result df by input info below

In [31]:
if re.match(r'^N\w{4,5}', 'N12345'):
    print('match')
else:
    print('not match')

match


In [37]:
def input_y_n(string):
    while True:
        t = input(string + ' Y/n\n')
        if t == 'Y' or t == 'n':
            break
        else:
            pass
    return t

In [48]:
def input_function(regex, string, intro='Format error. Please input again.\n'):
    while True:
        t = input(string)
        if re.match(regex, t):
            break
        else:
            print(intro)
            pass
    return t

In [53]:
input_function(r'[Yn]', 'Add the new tail num to the result? Y/n\n')

Add the new tail num to the result? Y/n
a
Format error. Please input again.

Add the new tail num to the result? Y/n
Y


'Y'

In [87]:
# This function can update result df by providing new result df
def update_result_by_input_info(result_df):
    regex_list = {
              'mfr_name': r'^\w+', 
              'model': r'^\w+', 
              'year': r'^\d{4}$|^Unknown$', 
              'cert_year': r'^\d{4}$|^Unknown$', 
              'eng_mfr_name': r'^\w+', 
              'eng_model':r'^\w+', 
              'aircraft_type': r'^Fixed Wing Multi-Engine$|^Fixed Wing Single-Engine$|^Rotorcraft$|^Unknown$'
                }
    key_list = list(regex_list.keys())
    column_list = ['tail_num']
    column_list.extend(key_list)
    while True:
        temp_result = pd.DataFrame(columns=column_list)
        tail_num = input_function(r'^N\w{4,5}$', 'Input the tail number:\n', intro='Tail number must in the form of Nxxxx or Nxxxxx\n')  
        if tail_num not in result_df['tail_num'].tolist():
            print('Tail num not found in the result df\n')
            i = input_function(r'[Yn]', 'Add the new tail num to the result? Y/n\n')
            # keep entering other info
            if i == 'n':
                t = input_function(r'[Yn]', 'Input another tail num? Y/n\n')
                if t == 'n':
                    return None
                else:
                    continue
            else:
                temp_result.loc[0, 'tail_num'] = tail_num
        else:
            temp_result = result_df[result_df['tail_num'] == tail_num].reset_index(drop=True)
        print('Current info for this tail number:\n')
        print(temp_result, '\n')
        # input other info
        for key in key_list:
            string1 = 'Do you want to update ' + key + ' ? Y/n\n'
            string2 = 'Input a new ' + key + ' or Unknown:\n'
            is_update = input_function(r'[Yn]', string1)
            if is_update == 'n':
                continue
            else:
                temp = input_function(regex_list[key], string2)
                temp_result.loc[0, key] = temp
        if isinstance(temp_result.loc[0, 'year'], str):
            if re.match(r'^\d{4}$', temp_result.loc[0, 'year']):
                temp_result.loc[0, 'year'] = int(temp_result.loc[0, 'year'])
            else:
                pass
        else:
            pass
        if isinstance(temp_result.loc[0, 'cert_year'], str):
            if re.match(r'^\d{4}$', temp_result.loc[0, 'cert_year']):
                temp_result.loc[0, 'cert_year'] = int(temp_result.loc[0, 'cert_year'])
            else:
                pass
        else:
            pass
        print(temp_result, '\n')
        is_update = input_function(r'[Yn]', 'Do you want to update this to result? Y/n\n')
        if is_update == 'Y':
            if tail_num in result_df['tail_num'].tolist():
                result_df = result_df.drop(result_df[result_df['tail_num'] == tail_num].index.values).reset_index(drop=True)
            else:
                pass
            result_df = pd.concat([result_df, temp_result], ignore_index=True)
        else:
            pass
#         mfr_name = input_function(r'^\w+', 'Input a new mfr name or Unknown:\n')
#         model = input_function(r'^\w+', 'Input a new model name or Unknown:\n')
#         year = input_function(r'^\d{4}$|^Unknown$', 'Input a new year or Unknown:\n')
#         cert_year = input_function(r'^\d{4}$|^Unknown$', 'Input a new cert year or Unknown:\n')
#         eng_mfr_name = input_function(r'^\w+', 'Input a new eng mfr name or Unknown:\n')
#         eng_model = input_function(r'^\w+', 'Input a new eng model or Unknown:\n')
#         aircraft_type = input_function(r'^Fixed Wing Multi-Engine$|^Fixed Wing Single-Engine$|^Rotorcraft$|^Unknown$', 
#                                    'Input a new aircraft type:\n'
#                                    'Fixed Wing Multi-Engine, Fixed Wing Single-Engine, Rotorcraft or Unknown:\n')
        
        # determine if want to add anothe tail number
        t = input_function(r'[Yn]', 'Input another tail num? Y/n\n')
        if t == 'n':
            result_df.to_csv('./tail_num/updated_all_tail_num.csv', index=False)
            return result_df
        else:
            continue
#         input()
    

In [80]:
int('2012')

2012

In [61]:
print(df[df['tail_num'] == 'N123AA'])

     tail_num          mfr_name     model    year  cert_year eng_mfr_name  \
4616   N123AA  AIRBUS INDUSTRIE  A320-214  2000.0     2000.0    CFM INTL.   

         eng_model            aircraft_type  
4616  CFM56 SERIES  Fixed Wing Multi-Engine  


In [67]:
eng_model = input_function(r'^Fixed Wing Multi-Engine$|^Fixed Wing Single-Engine$|^Rotorcraft$|^Unknown$', 
                                   'Input a new eng model:\n'
                                   'Fixed Wing Multi-Engine, Fixed Wing Single-Engine, Rotorcraft or Unknown:\n')

Input a new eng model:
Fixed Wing Multi-Engine, Fixed Wing Single-Engine, Rotorcraft or Unknown:
a
Format error. Please input again.

Input a new eng model:
Fixed Wing Multi-Engine, Fixed Wing Single-Engine, Rotorcraft or Unknown:
Fixed Wing Multi-Engine


In [68]:
eng_model

'Fixed Wing Multi-Engine'

In [72]:
update_result_by_input_info(df)

Input the tail number:
N123AA
Current info for this tail number:

     tail_num          mfr_name     model    year  cert_year eng_mfr_name  \
4616   N123AA  AIRBUS INDUSTRIE  A320-214  2000.0     2000.0    CFM INTL.   

         eng_model            aircraft_type  
4616  CFM56 SERIES  Fixed Wing Multi-Engine   

Do you want to update mfr_name ? Y/n
n
Do you want to update model ? Y/n
n
Do you want to update year ? Y/n
n
Do you want to update cert_year ? Y/n
n
Do you want to update eng_mfr_name ? Y/n
n
Do you want to update eng_model ? Y/n
n
Do you want to update aircraft_type ? Y/n
n
     tail_num          mfr_name     model    year  cert_year eng_mfr_name  \
4616   N123AA  AIRBUS INDUSTRIE  A320-214  2000.0     2000.0    CFM INTL.   

         eng_model            aircraft_type  
4616  CFM56 SERIES  Fixed Wing Multi-Engine   

Do you want to update this to result? Y/n
n
Input another tail num? Y/n
n


tail_num                       mfr_name           model    year  \
0      N3754A                         BOEING         737-832  2001.0   
1      N923DL  MCDONNELL DOUGLAS AIRCRAFT CO           MD-88  1988.0   
2      N551NW                         BOEING         757-251  2001.0   
3      N334NW               AIRBUS INDUSTRIE        A320-212  1992.0   
4      N675DL                         BOEING         757-232  1992.0   
5      N309US               AIRBUS INDUSTRIE        A320-211  1990.0   
6      N940DL  MCDONNELL DOUGLAS AIRCRAFT CO           MD-88  1989.0   
7      N920DL  MCDONNELL DOUGLAS AIRCRAFT CO           MD-88  1988.0   
8      N357NW               AIRBUS INDUSTRIE        A320-212  1998.0   
9      N199DN                         BOEING         767-332  1998.0   
10     N943AT                         BOEING         717-200  1999.0   
11     N3733Z                         BOEING         737-832  2000.0   
12     N925AT                         BOEING         717-200  2000.0   
13     N973DL  MCDONNELL DOUGLAS AIRCRAFT CO           MD-88  1991.0   
14     N907DE  MCDONNELL DOUGLAS CORPORATION           MD-88  1992.0   
15     N905DA              MCDONNELL DOUGLAS        MD-90-30  1995.0   
16     N392DA                         BOEING         737-832  2000.0   
17     N927DN                         BOEING        MD-90-30  1999.0   
18     N302NB               AIRBUS INDUSTRIE        A319-114  1999.0   
19     N966DL  MCDONNELL DOUGLAS AIRCRAFT CO           MD-88  1990.0   
20     N341NW               AIRBUS INDUSTRIE        A320-212  1992.0   
21     N970AT                         BOEING         717-200  2001.0   
22     N328NW               AIRBUS INDUSTRIE        A320-211  1992.0   
23     N608AT                         BOEING         717-200  2000.0   
24     N913DL  MCDONNELL DOUGLAS AIRCRAFT CO           MD-88  1988.0   
25     N378NW                         AIRBUS        A320-211  2003.0   
26     N703TW                         BOEING         757-2Q8  1996.0   
27     N927DA  MCDONNELL DOUGLAS AIRCRAFT CO           MD-88  1988.0   
28     N752AT                         BOEING         757-212  1984.0   
29     N915DL  MCDONNELL DOUGLAS AIRCRAFT CO           MD-88  1988.0   
...       ...                            ...             ...     ...   
5314   N651NK                         AIRBUS        A320-232  2017.0   
5315   N681NK                         AIRBUS        A321-231  2017.0   
5316   N652NK                         AIRBUS        A320-232  2017.0   
5317   N682NK                         AIRBUS        A321-231  2017.0   
5318   N282AK                         BOEING       737-900ER  2017.0   
5319   N283AK                         BOEING       737-900ER  2017.0   
5320   N887DN                         BOEING       737-900ER  2017.0   
5321   N329DN                         AIRBUS        A321-211  2017.0   
5322   N330DX                         AIRBUS        A321-211  2017.0   
5323   N331DN                         AIRBUS        A321-211  2017.0   
5324   N202HA                         AIRBUS       A321-271N  2017.0   
5325   N195SY                    EMBRAER S A  ERJ 170-200 LR  2017.0   
5326   N376AA                         BOEING         767-323  1992.0   
5327   N2333U                         BOEING       777-300ER  2017.0   
5328   N101NK               NAKAMURA HISASHI       TIERRA II     NaN   
5329   N2135U                         BOEING       777-300ER  2017.0   
5330   N59034                         CESSNA            210L     NaN   
5331   N2140U                         BOEING       777-300ER  2017.0   
5332   N2737U                         BOEING       777-300ER  2017.0   
5333   N2136U                         BOEING       777-300ER  2017.0   
5334   N878DA                          PIPER       PA-28-140     NaN   
5335   N2341U                         BOEING       777-300ER  2017.0   
5336   N2142U                         BOEING       777-300ER  2017.0   
5337   N2138U                         BOEING 

In [24]:
# tail nums not in the form of Nxxxxx 
df[df['tail_num'].str.match(r'^N\w{5}$').apply(lambda x: not x)]

tail_num          mfr_name     model    year  cert_year eng_mfr_name  \
65      N6701            BOEING   757-232  1999.0     1999.0        P & W   
177     N3765            BOEING   737-832  2001.0     2001.0    CFM INTL.   
236     N3768            BOEING   737-832  2002.0     2002.0    CFM INTL.   
293     N3767            BOEING   737-832  2001.0     2001.0    CFM INTL.   
353     N3759            BOEING   737-832  2001.0     2001.0    CFM INTL.   
439     N3753            BOEING   737-832  2001.0     2001.0    CFM INTL.   
449     N1608            BOEING   767-332  2000.0     2000.0           GE   
471     N704X            BOEING   757-2Q8  1997.0     2013.0        P & W   
501     N3766            BOEING   737-832  2001.0     2001.0    CFM INTL.   
515     N3756            BOEING   737-832  2001.0     2001.0    CFM INTL.   
590     N6700            BOEING   757-232  1999.0     1999.0        P & W   
609     N6709            BOEING   757-232  2000.0     2000.0        P & W   
624     N3752            BOEING   737-832  2001.0     2001.0    CFM INTL.   
632     N1609            BOEING   767-332  2000.0     2000.0           GE   
637     N6702            BOEING   757-232  1999.0     1999.0        P & W   
844     N1602            BOEING   767-332  1999.0     1999.0           GE   
1130    N744P  AIRBUS INDUSTRIE  A319-112  2000.0     2000.0    CFM INTL.   
1326    N1603            BOEING   767-332  1999.0     1999.0           GE   
1372    N1605            BOEING   767-332  1999.0     1999.0           GE   
3148    7820L            BOEING   737-79P  2002.0     2014.0     CFM INTL   
3557    7819A            BOEING   737-7Q8  2002.0     2014.0     CFM INTL   
5030   PLANET               NaN       NaN     NaN        NaN          NaN   

          eng_model            aircraft_type  
65           PW2037  Fixed Wing Multi-Engine  
177    CFM56 SERIES  Fixed Wing Multi-Engine  
236    CFM56 SERIES  Fixed Wing Multi-Engine  
293    CFM56 SERIES  Fixed Wing Multi-Engine  
353    CFM56 SERIES  Fixed Wing Multi-Engine  
439    CFM56 SERIES  Fixed Wing Multi-Engine  
449   CF6-80 SERIES  Fixed Wing Multi-Engine  
471          PW2037  Fixed Wing Multi-Engine  
501    CFM56 SERIES  Fixed Wing Multi-Engine  
515    CFM56 SERIES  Fixed Wing Multi-Engine  
590          PW2037  Fixed Wing Multi-Engine  
609          PW2037  Fixed Wing Multi-Engine  
624    CFM56 SERIES  Fixed Wing Multi-Engine  
632   CF6-80 SERIES  Fixed Wing Multi-Engine  
637          PW2037  Fixed Wing Multi-Engine  
844   CF6-80 SERIES  Fixed Wing Multi-Engine  
1130   CFM56 SERIES  Fixed Wing Multi-Engine  
1326  CF6-80 SERIES  Fixed Wing Multi-Engine  
1372  CF6-80 SERIES  Fixed Wing Multi-Engine  
3148     CFM56-7B24  Fixed Wing Multi-Engine  
3557     CFM56-7B24  Fixed Wing Multi-Engine  
5030            NaN                      NaN

In [65]:
df.groupby('aircraft_type').describe()

cert_year                                         \
                             count         mean       std     min     25%   
aircraft_type                                                               
Fixed Wing Multi-Engine     5084.0  2009.255114  6.843607  1986.0  2004.0   
Fixed Wing Single-Engine      21.0  2012.190476  4.501323  2004.0  2009.0   
Rotorcraft                     7.0  2016.000000  2.516611  2012.0  2014.5   

                                                    year               \
                             50%     75%     max   count         mean   
aircraft_type                                                           
Fixed Wing Multi-Engine   2011.0  2015.0  2018.0  4965.0  2004.756093   
Fixed Wing Single-Engine  2012.0  2016.0  2018.0    16.0  2006.000000   
Rotorcraft                2017.0  2018.0  2018.0     5.0  1990.600000   

                                                                             
                                std     min     25%     50%     75%     max  
aircraft_type                                                                
Fixed Wing Multi-Engine    7.489230  1977.0  2000.0  2004.0  2012.0  2017.0  
Fixed Wing Single-Engine  11.843423  1968.0  2004.0  2008.0  2013.5  2017.0  
Rotorcraft                15.026643  1980.0  1980.0  1980.0  2001.0  2012.0

# ^^^^^^^^^^^^^^^^^^^^^^^^^
# Update result df by input info above

In [ ]:
df_1 = pd.concat([df_1, scrape('N9401W')], ignore_index=True)

In [ ]:
len(tail_num_df['tail_num'].tolist())

In [ ]:
tail_num = 'N3HMAA'
result = scrape(find_match(tail_num, tail_num_convert))
result.loc[0, 'tail_num'] = tail_num
result

In [641]:
def scrape(tail_num):
    assert isinstance(tail_num, str)
    id_list = {'mfr_name': 'content_lbMfrName', 
              'model': 'content_Label7', 
              'year': 'content_Label17', 
              'cert_year': 'content_lbCertDate', 
              'eng_mfr_name': 'content_lbEngMfr', 
              'eng_model':'content_lbEngModel', 
              'aircraft_type': 'content_Label11'
             }
    key_list = list(id_list.keys())
    ret_id_list = {'mfr_name': 'content_drptrDeRegAircraft_lbDeRegMfrName_', 
              'model': 'content_drptrDeRegAircraft_lbDeRegModel_', 
              'year': 'content_drptrDeRegAircraft_lbDeRegYearMfr_', 
              'cert_year': 'content_drptrDeRegAircraft_lbDeRegCertDate_', 
              'eng_mfr_name': 'content_drptrDeRegAircraft_lbDREngMfr_', 
              'eng_model':'content_drptrDeRegAircraft_lbDREngModel_', 
                  }
    cert_cancel_id = 'content_drptrDeRegAircraft_lbDeRegCancelDate_'
    ret_key_list = list(ret_id_list.keys())
    
    
    column_list = ['tail_num']
    column_list.extend(key_list)
    df = pd.DataFrame(columns=column_list)
    df.loc[0, 'tail_num'] = tail_num
    response = requests.get(
        'https://registry.faa.gov/aircraftinquiry/NNum_Results.aspx'
        '?NNumbertxt=' + tail_num
    )
    soup = BeautifulSoup(response.content, "html.parser")
    not_retired = False
    for i in range(3):
        if soup.find('span', id=id_list[key_list[i]]):
            not_retired = True
        else:
            pass
    
    if not_retired:
        for key in key_list:
            s = soup.find('span', id=id_list[key]).string.strip()
            if s != 'None':
                df.loc[0, key] = s 
            else:
                pass
    else:
        j = 0
        while True:
            is_dereg = False
            for i in range(3):
                if soup.find('span', id=ret_id_list[ret_key_list[i]]+str(j)):
                    is_dereg = True
                else:
                    pass
            if not is_dereg:
                j -= 1
                break
            else:
                j += 1
                pass
        
        max_cancel_year = 1900
        # determine the latest cancelled cert
        if j != -1:
            max_j = -1
            for i in range(j+1):
                cancel_year = int(re.search(r'\w+/\w+/(\w+)', soup.find('span', id=cert_cancel_id + str(i)).string.strip()).group(1))
                if cancel_year > max_cancel_year:
                    max_cancel_year = cancel_year
                    max_j = i
        else:
            pass
        if j != -1 & max_cancel_year >= 2016:
#             for i in range(j+1):
            for key in ret_key_list:
                s = soup.find('span', id=ret_id_list[key]+str(max_j)).string.strip()
                if s != 'None':
                    df.loc[0, key] = s 
                else:
                    pass
        else:
            pass
    if isinstance(df.loc[0, 'cert_year'], str):
        df.loc[0, 'cert_year'] = re.search(r'\w+/\w+/(\w+)', df.loc[0, 'cert_year']).group(1).strip()
    return df

In [212]:
ret_id_list = {'mfr_name': 'content_drptrDeRegAircraft_lbDeRegMfrName_', 
              'model': 'content_drptrDeRegAircraft_lbDeRegModel_', 
              'year': 'content_drptrDeRegAircraft_lbDeRegYearMfr_', 
              'cert_year': 'content_drptrDeRegAircraft_lbDeRegCertDate_', 
              'eng_mfr_name': 'content_drptrDeRegAircraft_lbDREngMfr_', 
              'eng_model':'content_drptrDeRegAircraft_lbDREngModel_', 
                  }
ret_key_list = list(ret_id_list.keys())
i = 1
j = 1
ret_id_list[ret_key_list[i]]+str(j)

'content_drptrDeRegAircraft_lbDeRegModel_1'

In [531]:
scrape('N16545')

tail_num           mfr_name           model  year cert_year eng_mfr_name  \
0   N16545  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1990      2016        P & W   

  eng_model aircraft_type  
0     52053           NaN

In [204]:
response = requests.get(
        'https://registry.faa.gov/aircraftinquiry/NNum_Results.aspx'
        '?NNumbertxt=N214FR'
    )
soup = BeautifulSoup(response.content, "html.parser")

In [205]:
soup


<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<!--[if gte IE 8]>
   <meta http-equiv="X-UA-Compatible" content="IE=8">
  <![endif]-->
<!--[if lt IE 8]>
   <meta http-equiv="Content-Type" content="text/html; charset=iso-8859-1">
  <![endif]-->
<title>
	FAA Registry - Aircraft - N-Number Inquiry
</title><link href="css/master.css" media="all" rel="stylesheet" type="text/css"/>
<!-- User Styles -->
<link href="css/registry.css" media="all" rel="stylesheet" type="text/css"/><link href="css/project.css" media="all" rel="stylesheet" type="text/css"/>
<!--[if lte IE 6]>
   <link href="css/ie6.css" rel="stylesheet" type="text/css" media="screen,projection" />
  <![endif]-->
<!--[if IE 6]>
   <style type="text/css"> 
    html { overflow-y: hidden; }
    body { overflow-y: auto; }
    img#bg { position:absolute; z-index:-1; }
    #content { position:static; }
   </styl

In [40]:
print(scrape('N365AA'))

  tail_num    mfr_name  model  year cert_year eng_mfr_name eng_model  \
0   N365AA  AGUSTA SPA  A109E  2001      2014  P&W; CANADA    PW206C   

  aircraft_type  
0    Rotorcraft  


In [41]:
find_match('N365AA', tail_num_convert)

'N39365'

In [602]:
df_1 = pd.read_csv('./tail_num/new_all_tail_num.csv')
df_2 = pd.read_csv('./tail_num/tail_num_list.csv')

In [630]:
df_1['carrier'] = np.nan

In [631]:
for i in range(df_1.shape[0]):
    for j in range(df_2.shape[0]):
        if df_1.iloc[i, 0] == df_2.iloc[j, 0]:
            df_1.iloc[i, 8] = df_2.iloc[j, 1]
            break


In [642]:
rescrape('N678AA')

tail_num mfr_name model year cert_year eng_mfr_name eng_model aircraft_type
0   N678AA      NaN   NaN  NaN       NaN          NaN       NaN           NaN

In [639]:
tail_num_convert = pd.read_csv('./tail_num/updated_tail_num_convert.csv')

In [640]:
tail_num_convert[tail_num_convert['old_tail_num'] == 'N678AA']

Empty DataFrame
Columns: [old_tail_num, new_tail_num]
Index: []

In [593]:
tail_num_convert = update_convert_list(tail_num_convert, 'N956JB', 'N956JT')
tail_num_convert.to_csv('./tail_num/updated_tail_num_convert.csv', index=False)

In [717]:
df_aa_current = pd.read_csv('./tail_num/aa_current_fleet_tail_num_convert.csv')

In [730]:
df_aaus = df_aa_current[df_aa_current['old_tail_num'].str.match(r'^N\w{3}AA$').apply(lambda x: not x)]

In [734]:
df_aaaa = df_aa_current[df_aa_current['old_tail_num'].str.match(r'^N\w{3}AA$')]

In [736]:
df_aaus = df_aaus.reset_index(drop=True)

In [738]:
for i in range(df_aaus.shape[0]):
    df_aaus.loc[i, 'old_tail_num'] = df_aaus.loc[i, 'old_tail_num'][:-2] + 'AA'

In [740]:
df_aa_current = pd.concat([df_aaus, df_aaaa]).reset_index(drop=True)

In [746]:
df_aa_current = df_aa_current.sort_values(by=['old_tail_num']).reset_index(drop=True)

In [748]:
df_aa_current.to_csv('./tail_num/updated_aa_current_fleet_tail_num_convert.csv', index=False)

In [749]:
tail_num_convert = pd.read_csv('./tail_num/updated_tail_num_convert.csv')

In [752]:
for i in range(tail_num_convert.shape[0]):
    for j in range(df_aa_current.shape[0]):
        if tail_num_convert.iloc[i, 0] == df_aa_current.iloc[j, 0]:
            tail_num_convert.iloc[i, 1] = df_aa_current.iloc[j, 1]
            break
        

In [772]:
tail_num_convert[tail_num_convert['old_tail_num'] == 'N544AA']

old_tail_num new_tail_num
808       N544AA       N544UW

In [767]:
tail_num_convert.to_csv('./tail_num/updated_tail_num_convert.csv', index=False)

In [797]:
df_aa = df[df['tail_num'].str.match(r'^N\w{3}AA')]

In [798]:
df_aa = df_aa.reset_index(drop=True)

In [799]:
df_aa[df_aa['tail_num'] == 'N543AA']

tail_num mfr_name model  year  cert_year eng_mfr_name eng_model  \
799   N543AA      NaN   NaN   NaN        NaN          NaN       NaN   

    aircraft_type  
799           NaN

In [806]:
aa_list = df_aa['tail_num'].tolist()

In [819]:
aa_list[854]

'N817AA'

In [822]:
# rescrape AA tail num
count = 0
for tail_num in aa_list:
    times1 = datetime.now()
    df = pd.read_csv('./tail_num/new_all_tail_num.csv')
    rescrape(df, tail_num)
    count += 1
    print(tail_num, ',', count)
    times2 = datetime.now()
    print(' Time spent: '+ str(times2-times1)) # print the time cost

N4YBAA , 1
 Time spent: 0:00:00.879052
N434AA , 2
 Time spent: 0:00:00.910568
N541AA , 3
 Time spent: 0:00:00.952425
N489AA , 4
 Time spent: 0:00:01.031015
N439AA , 5
 Time spent: 0:00:00.768058
N468AA , 6
 Time spent: 0:00:00.766398
N569AA , 7
 Time spent: 0:00:00.901237
N466AA , 8
 Time spent: 0:00:01.014452
N501AA , 9
 Time spent: 0:00:01.328261
N438AA , 10
 Time spent: 0:00:01.205488
N426AA , 11
 Time spent: 0:00:01.080783
N566AA , 12
 Time spent: 0:00:01.011480
N499AA , 13
 Time spent: 0:00:01.372492
N4YPAA , 14
 Time spent: 0:00:00.935978
N520AA , 15
 Time spent: 0:00:00.856311
N547AA , 16
 Time spent: 0:00:00.905721
N4XBAA , 17
 Time spent: 0:00:00.852474
N504AA , 18
 Time spent: 0:00:01.012227
N4XPAA , 19
 Time spent: 0:00:00.939095
N4UCAA , 20
 Time spent: 0:00:01.094862
N514AA , 21
 Time spent: 0:00:01.009388
N471AA , 22
 Time spent: 0:00:01.194959
N4XVAA , 23
 Time spent: 0:00:00.861615
N4XGAA , 24
 Time spent: 0:00:01.091079
N4YEAA , 25
 Time spent: 0:00:00.939247
N565AA , 

N3DUAA , 204
 Time spent: 0:00:00.699172
N3ALAA , 205
 Time spent: 0:00:00.820531
N3EFAA , 206
 Time spent: 0:00:00.809300
N3EPAA , 207
 Time spent: 0:00:01.100039
N3JEAA , 208
 Time spent: 0:00:00.728556
N3NDAA , 209
 Time spent: 0:00:00.748764
N3HCAA , 210
 Time spent: 0:00:00.757971
N3ASAA , 211
 Time spent: 0:00:00.777655
N3HTAA , 212
 Time spent: 0:00:00.732042
N3EJAA , 213
 Time spent: 0:00:00.860918
N3HEAA , 214
 Time spent: 0:00:00.762960
N3KHAA , 215
 Time spent: 0:00:00.894378
N3KNAA , 216
 Time spent: 0:00:00.723254
N3DNAA , 217
 Time spent: 0:00:00.838247
N3GAAA , 218
 Time spent: 0:00:00.803375
N3CCAA , 219
 Time spent: 0:00:00.721670
N3AHAA , 220
 Time spent: 0:00:00.732545
N3DAAA , 221
 Time spent: 0:00:00.836950
N3GJAA , 222
 Time spent: 0:00:00.778534
N3ANAA , 223
 Time spent: 0:00:00.740604
N3KBAA , 224
 Time spent: 0:00:00.748272
N3FNAA , 225
 Time spent: 0:00:00.825913
N850AA , 226
 Time spent: 0:00:00.750990
N855AA , 227
 Time spent: 0:00:04.312688
N864AA , 228
 Ti

N7AUAA , 404
 Time spent: 0:00:00.736766
N3LWAA , 405
 Time spent: 0:00:00.770268
N3KGAA , 406
 Time spent: 0:00:00.754979
N796AA , 407
 Time spent: 0:00:00.794696
N799AA , 408
 Time spent: 0:00:00.765321
N797AA , 409
 Time spent: 0:00:00.738186
N793AA , 410
 Time spent: 0:00:00.752682
N784AA , 411
 Time spent: 0:00:00.815767
N5EVAA , 412
 Time spent: 0:00:00.761323
N7ASAA , 413
 Time spent: 0:00:00.724797
N5FDAA , 414
 Time spent: 0:00:00.805960
N5FRAA , 415
 Time spent: 0:00:00.750949
N7LLAA , 416
 Time spent: 0:00:00.809099
N4YDAA , 417
 Time spent: 0:00:00.746121
N513AA , 418
 Time spent: 0:00:00.725373
N4XSAA , 419
 Time spent: 0:00:00.859169
N4UBAA , 420
 Time spent: 0:00:00.716247
N425AA , 421
 Time spent: 0:00:00.762043
N4XCAA , 422
 Time spent: 0:00:00.806927
N571AA , 423
 Time spent: 0:00:00.841867
N3DLAA , 424
 Time spent: 0:00:00.768785
N5FLAA , 425
 Time spent: 0:00:01.121032
N5FKAA , 426
 Time spent: 0:00:00.800191
N5FBAA , 427
 Time spent: 0:00:00.726056
N5FAAA , 428
 Ti

N7CCAA , 604
 Time spent: 0:00:01.118612
N380AA , 605
 Time spent: 0:00:00.746008
N3NEAA , 606
 Time spent: 0:00:00.747928
N7LRAA , 607
 Time spent: 0:00:00.778517
N7BJAA , 608
 Time spent: 0:00:00.768668
N7BUAA , 609
 Time spent: 0:00:00.733152
N7LAAA , 610
 Time spent: 0:00:00.725362
N3NFAA , 611
 Time spent: 0:00:00.735255
N7BTAA , 612
 Time spent: 0:00:00.856611
N3BSAA , 613
 Time spent: 0:00:00.833492
N8APAA , 614
 Time spent: 0:00:00.988047
N3NVAA , 615
 Time spent: 0:00:00.797739
N892AA , 616
 Time spent: 0:00:00.745164
N894AA , 617
 Time spent: 0:00:00.814663
N890AA , 618
 Time spent: 0:00:00.804295
N897AA , 619
 Time spent: 0:00:00.771697
N896AA , 620
 Time spent: 0:00:00.813368
N989AA , 621
 Time spent: 0:00:00.749163
N893AA , 622
 Time spent: 0:00:00.743508
N891AA , 623
 Time spent: 0:00:00.753055
N987AA , 624
 Time spent: 0:00:01.028800
N991AA , 625
 Time spent: 0:00:00.940760
N993AA , 626
 Time spent: 0:00:01.008994
N889AA , 627
 Time spent: 0:00:01.055359
N990AA , 628
 Ti

N560AA , 804
 Time spent: 0:00:00.747767
N578AA , 805
 Time spent: 0:00:00.863796
N579AA , 806
 Time spent: 0:00:00.766920
N556AA , 807
 Time spent: 0:00:00.764839
N163AA , 808
 Time spent: 0:00:00.766580
N170AA , 809
 Time spent: 0:00:00.838792
N830AA , 810
 Time spent: 0:00:00.838533
N840AA , 811
 Time spent: 0:00:00.852858
N802AA , 812
 Time spent: 0:00:00.800240
N751AA , 813
 Time spent: 0:00:00.734154
N805AA , 814
 Time spent: 0:00:00.848084
N769AA , 815
 Time spent: 0:00:00.805659
N760AA , 816
 Time spent: 0:00:00.776919
N803AA , 817
 Time spent: 0:00:00.755278
N815AA , 818
 Time spent: 0:00:00.794094
N765AA , 819
 Time spent: 0:00:00.877765
N820AA , 820
 Time spent: 0:00:00.875656
N814AA , 821
 Time spent: 0:00:00.795880
N768AA , 822
 Time spent: 0:00:01.045335
N835AA , 823
 Time spent: 0:00:01.031518
N812AA , 824
 Time spent: 0:00:01.014278
N755AA , 825
 Time spent: 0:00:01.038641
N804AA , 826
 Time spent: 0:00:01.043528
N124AA , 827
 Time spent: 0:00:00.889078
N545AA , 828
 Ti

N903AA , 1004
 Time spent: 0:00:00.752404
N907AA , 1005
 Time spent: 0:00:00.814700
N928AA , 1006
 Time spent: 0:00:00.798639
N900AA , 1007
 Time spent: 0:00:00.752985
N909AA , 1008
 Time spent: 0:00:00.787958
N3PJAA , 1009
 Time spent: 0:00:00.806132
N3PLAA , 1010
 Time spent: 0:00:01.022968
N933AA , 1011
 Time spent: 0:00:00.781760
N3PGAA , 1012
 Time spent: 0:00:00.957438
N906AA , 1013
 Time spent: 0:00:00.747064
N910AA , 1014
 Time spent: 0:00:00.804718
N932AA , 1015
 Time spent: 0:00:00.775129
N3PUAA , 1016
 Time spent: 0:00:00.753804
N904AA , 1017
 Time spent: 0:00:00.766454
N8AWAA , 1018
 Time spent: 0:00:00.935564
N3PKAA , 1019
 Time spent: 0:00:01.000715
N3PEAA , 1020
 Time spent: 0:00:00.728078
N902AA , 1021
 Time spent: 0:00:00.775930
N8LNAA , 1022
 Time spent: 0:00:00.743846
N3RAAA , 1023
 Time spent: 0:00:00.976920
N3RBAA , 1024
 Time spent: 0:00:00.762326
N8LJAA , 1025
 Time spent: 0:00:00.724744
N8LLAA , 1026
 Time spent: 0:00:01.235875
N8LHAA , 1027
 Time spent: 0:00:02

In [878]:
tail_num_convert[tail_num_convert['old_tail_num'] == 'N399FR']

old_tail_num new_tail_num
543       N399FR       N399AN

In [784]:
rescrape(df, 'N508AA')

In [803]:
scrape('N508AY')

tail_num mfr_name     model  year cert_year eng_mfr_name eng_model  \
0   N508AY   AIRBUS  A321-231  2008      2015          IAE  V2533-A5   

             aircraft_type  
0  Fixed Wing Multi-Engine

# Cleaning

In [369]:
df = pd.read_csv('./tail_num/new_all_tail_num.csv')
tail_num_df = pd.read_csv('./tail_num/tail_num_list.csv')
df_with_carrier = df.merge(tail_num_df, on='tail_num', how='inner')

In [376]:
df_with_carrier.groupby(['eng_mfr_name', 'eng_model']).describe()

cert_year                                           \
                               count         mean        std     min      25%   
eng_mfr_name eng_model                                                          
CFM INTL     CFM56-3           145.0  2008.655172  11.016641  1990.0  1997.00   
             CFM56-5A          126.0  2009.166667   3.479655  1999.0  2009.00   
             CFM56-5B          442.0  2009.384615   6.948566  1998.0  2001.00   
             CFM56-7B         1655.0  2009.091843   6.167946  1998.0  2004.00   
             LEAP-1A            21.0  2016.904762   0.538958  2016.0  2017.00   
GE           CF34-10E          100.0  2011.000000   3.892495  2005.0  2007.00   
             CF34-3            230.0  2010.821739   6.533652  2000.0  2004.00   
             CF34-8C           203.0  2008.152709   4.641753  2001.0  2005.00   
             CF34-8E           107.0  2015.588785   1.008953  2014.0  2015.00   
             CF6-80C2          122.0  2003.254098   7.469565  1988.0  1999.00   
             CF6-80E1           10.0  2016.200000   0.421637  2016.0  2016.00   
             GE90-110B1         10.0  2009.000000   0.666667  2008.0  2009.00   
             GE90-115B          35.0  2015.200000   1.641377  2013.0  2014.00   
             GE90-90B           22.0  2006.863636   8.402355  1998.0  1999.00   
             GENX-1B70          32.0  2014.500000   1.545023  2012.0  2013.75   
             GENX-1B74/75/      35.0  2015.942857   0.968409  2014.0  2015.00   
IAE          V2522-A5           55.0  2012.200000   2.882900  2002.0  2013.00   
             V2524-A5          117.0  2009.555556   5.878723  2004.0  2004.00   
             V2527-A5          339.0  2009.233038   5.206559  1998.0  2004.00   
             V2528-D5           65.0  2007.923077   7.226919  1995.0  2009.00   
             V2533-A5          333.0  2015.081081   1.285708  2009.0  2015.00   
P & W        JT8D-217           44.0  2011.113636  11.135980  1986.0  2012.00   
             JT8D-219          167.0  2002.610778  11.418570  1987.0  1992.00   
             PW1127G-JM          5.0  2016.000000   0.000000  2016.0  2016.00   
             PW1133G-JM          1.0  2017.000000        NaN  2017.0  2017.00   
             PW2037            119.0  2007.428571   8.335722  1991.0  1999.50   
             PW2040             30.0  2010.533333   4.516127  1992.0  2009.00   
             PW4056             29.0  2013.034483   4.153164  2000.0  2013.00   
             PW4060             76.0  2006.631579   8.485033  1990.0  1998.00   
             PW4077             19.0  2013.052632   0.229416  2013.0  2013.00   
             PW4090             33.0  2011.424242   3.799920  2001.0  2013.00   
             PW4168A            50.0  2009.400000   2.740475  2005.0  2009.00   
ROLLS-ROYC   AE3007A           187.0  2007.828877   6.609103  2000.0  2003.00   
             BR700-715         111.0  2005.216216   5.254272  2000.0  2001.00   
             RB211-535E4        35.0  2012.457143   8.226009  1986.0  2015.00   
             RB211-535E4B       83.0  2006.493976   8.292479  1993.0  1999.00   
             RB211-535E4C       21.0  2002.476190   1.400680  2001.0  2001.00   
             TRENT 772B-60      54.0  2014.444444   1.283666  2011.0  2014.00   
             TRENT 892          47.0  2001.170213   3.009544  1999.0  2000.00   
             TRENT 895           8.0  1999.375000   1.060660  1999.0  1999.00   

                                                      year               \
                               50%     75%     max   count         mean   
eng_mfr_name eng_model                                                    
CFM INTL     CFM56-3        2017.0  2018.0  2018.0   150.0  1993.766667   
             CFM56-5A       2009.0  2009.0  2016.0   126.0  1997.841270   
             CFM56-5B       2013.0  2015.0  2018.0   443.0  2006.137698   
             CFM56-7B       2010.0  2015.0  2018.0  1655.0  2007.819335   
             LEAP-

In [371]:
df_1 = df_with_carrier[df_with_carrier.iloc[:, 1:8].isnull().all(axis=1).apply(lambda x: not x)]

In [372]:
df_1[df_1['eng_mfr_name'] == 'P & W'].groupby('eng_model').describe()

cert_year                                                          \
               count         mean        std     min     25%     50%     75%   
eng_model                                                                      
JT8D-217        44.0  2011.113636  11.135980  1986.0  2012.0  2017.0  2017.0   
JT8D-219       167.0  2002.610778  11.418570  1987.0  1992.0  1999.0  2016.0   
PW1127G-JM       5.0  2016.000000   0.000000  2016.0  2016.0  2016.0  2016.0   
PW1133G-JM       1.0  2017.000000        NaN  2017.0  2017.0  2017.0  2017.0   
PW2037         119.0  2007.428571   8.335722  1991.0  1999.5  2009.0  2015.0   
PW2040          30.0  2010.533333   4.516127  1992.0  2009.0  2009.0  2013.0   
PW4056          29.0  2013.034483   4.153164  2000.0  2013.0  2013.0  2016.0   
PW4060          76.0  2006.631579   8.485033  1990.0  1998.0  2008.5  2013.0   
PW4077          19.0  2013.052632   0.229416  2013.0  2013.0  2013.0  2013.0   
PW4090          33.0  2011.424242   3.799920  2001.0  2013.0  2013.0  2013.0   
PW4168A         50.0  2009.400000   2.740475  2005.0  2009.0  2009.0  2009.0   

                     year                                                  \
               max  count         mean       std     min      25%     50%   
eng_model                                                                   
JT8D-217    2018.0   44.0  1988.431818  1.264869  1986.0  1988.00  1988.0   
JT8D-219    2018.0  167.0  1991.556886  3.787431  1987.0  1989.00  1991.0   
PW1127G-JM  2016.0    5.0  2016.000000  0.000000  2016.0  2016.00  2016.0   
PW1133G-JM  2017.0    1.0  2017.000000       NaN  2017.0  2017.00  2017.0   
PW2037      2018.0  119.0  1996.134454  4.597207  1984.0  1992.50  1997.0   
PW2040      2017.0   30.0  1997.200000  5.909665  1989.0  1991.25  2002.0   
PW4056      2018.0   29.0  1993.965517  4.338225  1988.0  1990.00  1991.0   
PW4060      2018.0   76.0  1995.421053  3.760366  1990.0  1992.00  1995.5   
PW4077      2014.0   19.0  1996.894737  2.208437  1995.0  1995.00  1996.0   
PW4090      2013.0   33.0  1999.060606  1.951592  1997.0  1997.00  1999.0   
PW4168A     2015.0   50.0  2003.580000  2.524088  2000.0  2001.00  2004.0   

                            
               75%     max  
eng_model                   
JT8D-217    1989.0  1991.0  
JT8D-219    1992.5  1999.0  
PW1127G-JM  2016.0  2016.0  
PW1133G-JM  2017.0  2017.0  
PW2037      1999.0  2005.0  
PW2040      2003.0  2003.0  
PW4056      1999.0  2000.0  
PW4060      1998.0  2003.0  
PW4077      2000.0  2000.0  
PW4090      2001.0  2002.0  
PW4168A     2006.0  2007.0

In [374]:
df_1[(df_1['eng_model'] == 'JT8D SERIES') & (df_1['eng_mfr_name'] == 'P & W')]
# df_1[(df_1['eng_model'] == 'CFM56 SERIES') & (df_1['model'] == 'A319-114')]
# df_1[(df_1['model'] == 'A319-115')]

Empty DataFrame
Columns: [tail_num, mfr_name, model, year, cert_year, eng_mfr_name, eng_model, aircraft_type, carrier]
Index: []

In [375]:
# df_1[(df_1['eng_model'] == 'JT8D SERIES') & (df_1['model'] == 'MD-88')]
df_1[(df_1['model'] == 'DC-9-82(MD-82)') & (df_1['eng_mfr_name'] == 'P & W')]

tail_num           mfr_name           model    year  cert_year  \
4160   N541AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1990.0     2014.0   
4161   N489AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1989.0     2018.0   
4163   N468AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1988.0     2017.0   
4165   N466AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1988.0     2017.0   
4166   N501AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1989.0     1989.0   
4168   N426AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1986.0     1986.0   
4170   N499AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1989.0     2017.0   
4172   N520AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1990.0     2017.0   
4173   N547AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1990.0     2012.0   
4175   N504AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1989.0     2018.0   
4177   N4UCAA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1987.0     2011.0   
4178   N514AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1990.0     2018.0   
4179   N471AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1988.0     2017.0   
4281   N480AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1988.0     2017.0   
4285   N482AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1988.0     2018.0   
4287   N473AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1988.0     2017.0   
4291   N502AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1989.0     1989.0   
4298   N477AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1988.0     2017.0   
4300   N484AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1988.0     2017.0   
4301   N403AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1986.0     2017.0   
4303   N486AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1988.0     2017.0   
4305   N467AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1988.0     2017.0   
4309   N479AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1988.0     2017.0   
4311   N474AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1988.0     2016.0   
4530   N487AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1988.0     2018.0   
4543   N496AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1989.0     2016.0   
4545   N472AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1988.0     2017.0   
4547   N493AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1989.0     2017.0   
4548   N555AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1991.0     2012.0   
4549   N485AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1988.0     2017.0   
4569   N513AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1990.0     2013.0   
4571   N4UBAA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1987.0     2018.0   
4572   N425AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1986.0     1987.0   
4633   N469AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1988.0     2017.0   
4635   N481AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1988.0     2017.0   
4636   N550AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1991.0     2012.0   
4637   N470AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1988.0     2017.0   
4653   N424AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1986.0     1986.0   
4654   N488AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1988.0     1989.0   
4684   N528AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1990.0     1990.0   
4687   N475AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1988.0     2017.0   
4688   N483AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1988.0     2018.0   
4690   N494AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1989.0     1989.0   
5196   N548AA  MCDONNELL DOUGLAS  DC-9-82(MD-82)  1991.0     2012.0   

     eng_mfr_name eng_model            aircraft_type carrier  
4160        P & W  JT8D-217  Fixed Wing Multi-Engine      AA  
4161        P & W  JT8D-217  Fixed Wing Multi-Engine      AA  
4163        P & W  JT8D-217                      NaN      AA  
4165        P & W  JT8D-217                      NaN      AA  
4166        P & W  JT8D-217  Fixed Wing Multi-Engine      AA  
4168        P & W  JT8D-217  Fixed Wing Multi-Engine      AA  
4170        P & W  JT8D-217  Fixed Wing Multi-Engine      AA  
4172        P & W  JT8D-217  Fixed Wing Multi-Engine      AA  
4173        P & W  JT8D-217  Fixed Wing Multi-Engine      AA  
4175        P & W  JT8D-217                      NaN      AA  
4177        P & W  JT8D-217  Fixed Wing Multi-Engine      AA  
4178        P & W  JT8D-217                      NaN 

In [363]:
df['model'] = df['model'].str.replace('MD 83', 'DC-9-83(MD-83)', regex=False)

In [204]:
df.loc[2403, 'eng_model'] = 'RB211-535E4B'

In [192]:
#  & (df_1['carrier'] == 'AA')
# df_1[(df_1['model'] == 'A319-112')].groupby('carrier').describe()
df_1[(df_1['model'] == '737-7H4')]

tail_num mfr_name    model    year  cert_year eng_mfr_name     eng_model  \
2805   N250WN   BOEING  737-7H4  2006.0     2006.0     CFM INTL       CFM56-7   
2806   N263WN   BOEING  737-7H4  2007.0     2007.0     CFM INTL       CFM56-7   
2807   N466WN   BOEING  737-7H4  2004.0     2010.0     CFM INTL  CFM56 SERIES   
2808   N457WN   BOEING  737-7H4  2004.0     2004.0     CFM INTL  CFM56 SERIES   
2816   N446WN   BOEING  737-7H4  2003.0     2003.0     CFM INTL  CFM56 SERIES   
2818   N419WN   BOEING  737-7H4  2001.0     2002.0     CFM INTL  CFM56 SERIES   
2819   N420WN   BOEING  737-7H4  2001.0     2002.0     CFM INTL  CFM56 SERIES   
2820   N460WN   BOEING  737-7H4  2004.0     2004.0     CFM INTL  CFM56 SERIES   
2822   N445WN   BOEING  737-7H4  2003.0     2004.0     CFM INTL  CFM56 SERIES   
2823   N233LV   BOEING  737-7H4  2006.0     2006.0     CFM INTL  CFM56 SERIES   
2824   N963WN   BOEING  737-7H4  2011.0     2011.0     CFM INTL       CFM56-7   
2827   N465WN   BOEING  737-7H4  2004.0     2004.0     CFM INTL  CFM56 SERIES   
2828   N289CT   BOEING  737-7H4  2007.0     2007.0     CFM INTL       CFM56-7   
2829   N785SW   BOEING  737-7H4  2000.0     2000.0     CFM INTL  CFM56 SERIES   
2830   N737JW   BOEING  737-7H4  1999.0     1999.0     CFM INTL  CFM56 SERIES   
2831   N227WN   BOEING  737-7H4  2005.0     2006.0     CFM INTL  CFM56 SERIES   
2832   N406WN   BOEING  737-7H4  2001.0     2001.0     CFM INTL  CFM56 SERIES   
2835   N958WN   BOEING  737-7H4  2011.0     2011.0     CFM INTL       CFM56-7   
2836   N438WN   BOEING  737-7H4  2003.0     2003.0     CFM INTL  CFM56 SERIES   
2838   N463WN   BOEING  737-7H4  2004.0     2004.0     CFM INTL  CFM56 SERIES   
2839   N787SA   BOEING  737-7H4  2000.0     2001.0     CFM INTL  CFM56 SERIES   
2840   N221WN   BOEING  737-7H4  2005.0     2005.0     CFM INTL  CFM56 SERIES   
2841   N208WN   BOEING  737-7H4  2005.0     2005.0     CFM INTL  CFM56 SERIES   
2842   N461WN   BOEING  737-7H4  2004.0     2004.0     CFM INTL  CFM56 SERIES   
2844   N764SW   BOEING  737-7H4  2000.0     2000.0     CFM INTL  CFM56 SERIES   
2846   N491WN   BOEING  737-7H4  2004.0     2004.0     CFM INTL  CFM56 SERIES   
2850   N215WN   BOEING  737-7H4  2005.0     2005.0     CFM INTL  CFM56 SERIES   
2851   N210WN   BOEING  737-7H4  2005.0     2005.0     CFM INTL  CFM56 SERIES   
2852   N206WN   BOEING  737-7H4  2005.0     2005.0     CFM INTL  CFM56 SERIES   
2855   N260WN   BOEING  737-7H4  2006.0     2007.0     CFM INTL       CFM56-7   
...       ...      ...      ...     ...        ...          ...           ...   
3416   N283WN   BOEING  737-7H4  2007.0     2007.0     CFM INTL       CFM56-7   
3418   N228WN   BOEING  737-7H4  2005.0     2006.0     CFM INTL  CFM56 SERIES   
3419   N758SW   BOEING  737-7H4  1999.0     1999.0     CFM INTL  CFM56 SERIES   
3421   N252WN   BOEING  737-7H4  2006.0     2006.0     CFM INTL       CFM56-7   
3424   N219WN   BOEING  737-7H4  2005.0     2005.0     CFM INTL  CFM56 SERIES   
3433   N278WN   BOEING  737-7H4  2007.0     2007.0     CFM INTL       CFM56-7   
3434   N775SW   BOEING  737-7H4  2000.0     2000.0     CFM INTL  CFM56 SERIES   
3436   N214WN   BOEING  737-7H4  2005.0     2005.0     CFM INTL  CFM56 SERIES   
3438   N740SW   BOEING  737-7H4  1998.0     1999.0     CFM INTL  CFM56 SERIES   
3442   N230WN   BOEING  737-7H4  2006.0     2006.0     CFM INTL  CFM56 SERIES   
3445   N909WN   BOEING  737-7H4  2008.0     2008.0     CFM INTL       CFM56-7   
3446   N748SW   BOEING  737-7H4  1999.0     1999.0     CFM INTL  CFM56 SERIES   
3448   N955WN   BOEING  737-7H4  2011.0     2011.0     CFM INTL       CFM56-7   
3452   N907WN   BOEING  737-7H4  2008.0     2008.0     CFM INTL       CFM56-7   
3453   N455WN   BOEING  737-7H4  2004.0     2004.0     CFM INTL  CFM56 SERIES   
3455   N704SW   BOEING  737-7H4  1998.0     1998.0     CFM INTL  CFM56 SERIES   
3457   N743SW   BOEING  737-7H4  1998.0     1999.0     CFM INTL  CFM56 SERIES   
3458   N454WN   BOEING  737-7H4  200

In [1540]:
aa_df = df_1[df_1['carrier'] == 'AA']

In [1472]:
aa_df[aa_df['tail_num'].str.match(r'N76[89]UA$')]

tail_num mfr_name      model    year  cert_year eng_mfr_name   eng_model  \
2652   N768UA   BOEING  777-222ER  1995.0     2013.0        P & W  PW4000 SER   
2800   N769UA   BOEING  777-222ER  1995.0     2013.0        P & W  PW4000 SER   

                aircraft_type carrier  
2652  Fixed Wing Multi-Engine      UA  
2800  Fixed Wing Multi-Engine      UA

In [1535]:
dl_763 = dl_df[dl_df['tail_num'].str.match(r'N140LL$')].index.tolist()

In [1536]:
dl_763.extend(dl_df[dl_df['tail_num'].str.match(r'N1402A$')].index.tolist())

In [1543]:
dl_df.loc[dl_763]

tail_num mfr_name    model    year  cert_year eng_mfr_name   eng_model  \
340   N140LL   BOEING  767-332  1993.0     2011.0        P & W  PW4000 SER   
160   N1402A   BOEING  767-332  1993.0     2011.0        P & W  PW4000 SER   

               aircraft_type carrier  
340  Fixed Wing Multi-Engine      DL  
160  Fixed Wing Multi-Engine      DL

In [105]:
df[(df['model'] == '717-200') & (df['eng_mfr_name'] == 'ROLLS-ROYC')]

tail_num mfr_name    model    year  cert_year eng_mfr_name  \
10     N943AT   BOEING  717-200  1999.0     2014.0   ROLLS-ROYC   
12     N925AT   BOEING  717-200  2000.0     2001.0   ROLLS-ROYC   
20     N970AT   BOEING  717-200  2001.0     2002.0   ROLLS-ROYC   
22     N608AT   BOEING  717-200  2000.0     2009.0   ROLLS-ROYC   
43     N971AT   BOEING  717-200  2001.0     2002.0   ROLLS-ROYC   
45     N603AT   BOEING  717-200  2001.0     2001.0   ROLLS-ROYC   
47     N959AT   BOEING  717-200  2001.0     2001.0   ROLLS-ROYC   
51     N924AT   BOEING  717-200  2000.0     2001.0   ROLLS-ROYC   
58     N982AT   BOEING  717-200  2002.0     2003.0   ROLLS-ROYC   
61     N989AT   BOEING  717-200  2001.0     2002.0   ROLLS-ROYC   
65     N910AT   BOEING  717-200  2001.0     2001.0   ROLLS-ROYC   
68     N933AT   BOEING  717-200  2000.0     2000.0   ROLLS-ROYC   
73     N994AT   BOEING  717-200  2002.0     2002.0   ROLLS-ROYC   
79     N896AT   BOEING  717-200  2005.0     2005.0   ROLLS-ROYC   
87     N920AT   BOEING  717-200  2000.0     2001.0   ROLLS-ROYC   
121    N993AT   BOEING  717-200  2002.0     2002.0   ROLLS-ROYC   
132    N948AT   BOEING  717-200  1999.0     2014.0   ROLLS-ROYC   
134    N944AT   BOEING  717-200  1999.0     2014.0   ROLLS-ROYC   
138    N928AT   BOEING  717-200  2000.0     2000.0   ROLLS-ROYC   
146    N947AT   BOEING  717-200  1999.0     2014.0   ROLLS-ROYC   
151    N934AT   BOEING  717-200  2000.0     2000.0   ROLLS-ROYC   
166    N942AT   BOEING  717-200  1999.0     2014.0   ROLLS-ROYC   
187    N981AT   BOEING  717-200  2002.0     2003.0   ROLLS-ROYC   
192    N919AT   BOEING  717-200  2001.0     2001.0   ROLLS-ROYC   
195    N937AT   BOEING  717-200  2001.0     2001.0   ROLLS-ROYC   
197    N967AT   BOEING  717-200  2001.0     2001.0   ROLLS-ROYC   
208    N906AT   BOEING  717-200  2001.0     2001.0   ROLLS-ROYC   
209    N972AT   BOEING  717-200  2001.0     2002.0   ROLLS-ROYC   
241    N985AT   BOEING  717-200  2001.0     2001.0   ROLLS-ROYC   
257    N946AT   BOEING  717-200  1999.0     2014.0   ROLLS-ROYC   
...       ...      ...      ...     ...        ...          ...   
1225   N995AT   BOEING  717-200  2002.0     2002.0   ROLLS-ROYC   
1261   N929AT   BOEING  717-200  2000.0     2000.0   ROLLS-ROYC   
1318   N953AT   BOEING  717-200  2000.0     2000.0   ROLLS-ROYC   
1710   N478HA   BOEING  717-200  2001.0     2011.0   ROLLS-ROYC   
1711   N490HA   BOEING  717-200  2000.0     2011.0   ROLLS-ROYC   
1712   N477HA   BOEING  717-200  2001.0     2011.0   ROLLS-ROYC   
1713   N485HA   BOEING  717-200  2001.0     2011.0   ROLLS-ROYC   
1714   N481HA   BOEING  717-200  2001.0     2011.0   ROLLS-ROYC   
1715   N486HA   BOEING  717-200  2001.0     2011.0   ROLLS-ROYC   
1716   N491HA   BOEING  717-200  2003.0     2012.0   ROLLS-ROYC   
1717   N476HA   BOEING  717-200  2001.0     2011.0   ROLLS-ROYC   
1718   N493HA   BOEING  717-200  2005.0     2011.0   ROLLS-ROYC   
1719   N487HA   BOEING  717-200  2001.0     2011.0   ROLLS-ROYC   
1720   N480HA   BOEING  717-200  2001.0     2011.0   ROLLS-ROYC   
1721   N492HA   BOEING  717-200  2004.0     2012.0   ROLLS-ROYC   
1722   N479HA   BOEING  717-200  2001.0     2011.0   ROLLS-ROYC   
1723   N484HA   BOEING  717-200  2001.0     2011.0   ROLLS-ROYC   
1724   N489HA   BOEING  717-200  1998.0     2011.0   ROLLS-ROYC   
1725   N488HA   BOEING  717-200  1998.0     2011.0   ROLLS-ROYC   
1726   N475HA   BOEING  717-200  2001.0     2011.0   ROLLS-ROYC   
1727   N483HA   BOEING  717-200  2001.0     2011.0   ROLLS-ROYC   
3514   N945AT   BOEING  717-200  1999.0     2014.0   ROLLS-ROYC   
3541   N892AT   BOEING  717-200  2004.0     2004.0   ROLLS-ROYC   
3614   N989DN   BOEING  717-200  2001.0     2015.0   ROLLS-ROYC   
3629   N987DN   BOEING  717-200  2000.0     2016.0   ROLLS-ROYC   
3645   N988DN   BOEING  717-200  2001.0     2015.0   ROLLS-ROYC   
3945   N494HA   BOEING  717-200  2004.0     2016.0   ROLLS-ROYC   
3965   N495HA   BOEING  717-200  2004.0     2017.0  

In [1558]:
df[(df['eng_model'] == 'PW1133G-JM') & (df['eng_mfr_name'] == 'P & W')]

tail_num mfr_name      model    year  cert_year eng_mfr_name   eng_model  \
4137   N202HA   AIRBUS  A321-271N  2017.0     2017.0        P & W  PW1133G-JM   

                aircraft_type  
4137  Fixed Wing Multi-Engine

In [211]:
df[(df['model'].str.match('^737-3.*')) & (df['eng_mfr_name'] == 'CFM INTL')]

tail_num mfr_name    model    year  cert_year eng_mfr_name     eng_model  \
2803   N687SW   BOEING  737-3Q8  1986.0     2015.0     CFM INTL       CFM56-3   
2810   N365SW   BOEING  737-3H4  1993.0     2004.0     CFM INTL       CFM56-3   
2811   N608SW   BOEING  737-3H4  1995.0     2018.0     CFM INTL       CFM56-3   
2813   N654SW   BOEING  737-3H4  1997.0     2018.0     CFM INTL       CFM56-3   
2814   N387SW   BOEING  737-3H4  1994.0     2018.0     CFM INTL       CFM56-3   
2815   N627SW   BOEING  737-3H4  1996.0     2010.0     CFM INTL       CFM56-3   
2825   N628SW   BOEING  737-3H4  1996.0     2018.0     CFM INTL       CFM56-3   
2826   N637SW   BOEING  737-3H4  1996.0     2018.0     CFM INTL       CFM56-3   
2837   N353SW   BOEING  737-3H4  1990.0     1990.0     CFM INTL       CFM56-3   
2847   N638SW   BOEING  737-3H4  1996.0        NaN     CFM INTL       CFM56-3   
2848   N391SW   BOEING  737-3H4  1994.0     2018.0     CFM INTL       CFM56-3   
2849   N658SW   BOEING  737-3L9  1985.0     1998.0     CFM INTL  CFM56 SERIES   
2853   N360SW   BOEING  737-3H4  1992.0     2017.0     CFM INTL       CFM56-3   
2854   N641SW   BOEING  737-3H4  1996.0     2018.0     CFM INTL       CFM56-3   
2875   N340LV   BOEING  737-3K2  1987.0     2003.0     CFM INTL  CFM56 SERIES   
2889   N652SW   BOEING  737-3H4  1997.0     1997.0     CFM INTL       CFM56-3   
2904   N380SW   BOEING  737-3H4  1994.0     2004.0     CFM INTL       CFM56-3   
2907   N633SW   BOEING  737-3H4  1996.0     2018.0     CFM INTL       CFM56-3   
2914   N645SW   BOEING  737-3H4  1997.0     1997.0     CFM INTL       CFM56-3   
2915   N317WN   BOEING  737-3Q8  1988.0     1999.0     CFM INTL       CFM56-3   
2916   N697SW   BOEING  737-3TO  1988.0     1995.0     CFM INTL  CFM56 SERIES   
2920   N646SW   BOEING  737-3H4  1997.0     1997.0     CFM INTL       CFM56-3   
2921   N619SW   BOEING  737-3H4  1995.0     2017.0     CFM INTL       CFM56-3   
2926   N644SW   BOEING  737-3H4  1997.0     2018.0     CFM INTL       CFM56-3   
2942   N617SW   BOEING  737-3H4  1995.0     2018.0     CFM INTL       CFM56-3   
2946   N386SW   BOEING  737-3H4  1994.0     2018.0     CFM INTL       CFM56-3   
2950   N356SW   BOEING  737-3H4  1991.0     1991.0     CFM INTL       CFM56-3   
2958   N364SW   BOEING  737-3H4  1993.0     2004.0     CFM INTL       CFM56-3   
2962   N606SW   BOEING  737-3H4  1995.0     2017.0     CFM INTL       CFM56-3   
2976   N601WN   BOEING  737-3H4  1995.0     2018.0     CFM INTL       CFM56-3   
...       ...      ...      ...     ...        ...          ...           ...   
3423   N642WN   BOEING  737-3H4  1997.0     2017.0     CFM INTL       CFM56-3   
3425   N378SW   BOEING  737-3H4  1994.0     1994.0     CFM INTL       CFM56-3   
3426   N631SW   BOEING  737-3H4  1996.0     2018.0     CFM INTL       CFM56-3   
3427   N684WN   BOEING  737-3TO  1988.0     2013.0     CFM INTL  CFM56 SERIES   
3430   N398SW   BOEING  737-3H4  1995.0     2018.0     CFM INTL       CFM56-3   
3432   N345SA   BOEING  737-3K2  1987.0     2016.0     CFM INTL  CFM56 SERIES   
3440   N621SW   BOEING  737-3H4  1996.0     2018.0     CFM INTL       CFM56-3   
3441   N665WN   BOEING  737-3Y0  1986.0     2012.0     CFM INTL  CFM56 SERIES   
3443   N369SW   BOEING  737-3H4  1993.0     1993.0     CFM INTL       CFM56-3   
3444   N375SW   BOEING  737-3H4  1993.0     1993.0     CFM INTL       CFM56-3   
3447   N379SW   BOEING  737-3H4  1994.0     1994.0     CFM INTL       CFM56-3   
3450   N635SW   BOEING  737-3H4  1996.0     2018.0     CFM INTL       CFM56-3   
3451   N384SW   BOEING  737-3H4  1994.0     2018.0     CFM INTL       CFM56-3   
3456   N610WN   BOEING  737-3H4  1995.0     1995.0     CFM INTL       CFM56-3   
3459   N607SW   BOEING  737-3H4  1995.0     2018.0     CFM INTL       CFM56-3   
3463   N397SW   BOEING  737-3H4  1995.0     2018.0     CFM INTL       CFM56-3   
3465   N362SW   BOEING  737-3H4  1992.0     2017.0     CFM INTL       CFM56-3   
3466   N390SW   BOEING  737-3H4  199

In [367]:
idx_lst = df[(df['model'] == 'MD-88') & (df['eng_mfr_name'] == 'P & W')].index.tolist()
for i in idx_lst:
#     df.loc[i, 'model'] = df.loc[i, 'model'] + 'ER'
    df.loc[i, 'eng_model'] = 'JT8D-219'
    
#     df.loc[i, 'eng_model'] = 'CFM56 SERIES'

In [1544]:
for i in dl_763:
    df.loc[i, 'model'] = df.loc[i, 'model'][:-2]
#     df.loc[i, 'eng_model'] = 'CFM56 SERIES'

In [1224]:
idx_lst

tail_num mfr_name     model    year  cert_year eng_mfr_name eng_model  \
596    N311US   AIRBUS  A320-211  1990.0     2009.0           GE     30016   
2914   N645SW   BOEING   737-3H4  1997.0     1997.0           GE     30016   
2915   N317WN   BOEING   737-3Q8  1988.0     1999.0           GE     30016   
2920   N646SW   BOEING   737-3H4  1997.0     1997.0           GE     30016   
3441   N665WN   BOEING   737-3Y0  1986.0     2012.0           GE     30016   
5193   N636WN   BOEING   737-3H4  1996.0        NaN           GE     30016   

     aircraft_type  
596            NaN  
2914           NaN  
2915           NaN  
2920           NaN  
3441           NaN  
5193           NaN

In [1014]:
tail_num_convert = pd.read_csv('./tail_num/updated_tail_num_convert.csv')
tail_num_convert[tail_num_convert['old_tail_num'] == 'N4WNAA']

old_tail_num new_tail_num
120       N4WNAA       N9405T

In [ ]:
tail_num_convert = update_convert_list(tail_num_convert, '', '')
tail_num_convert.to_csv('./tail_num/updated_tail_num_convert.csv', index=False)

In [1077]:
df['eng_mfr_name'] = df['eng_mfr_name'].str.replace('ALLISON', 'ROLLS-ROYC', regex=False)

In [368]:
df.to_csv('./tail_num/new_all_tail_num.csv', index=False)

In [1169]:
df = update_result_by_input_info(df)
df.to_csv('./tail_num/new_all_tail_num.csv', index=False)

Input the tail number:
N841VA
Current info for this tail number:

  tail_num mfr_name     model    year  cert_year eng_mfr_name eng_model  \
0   N841VA   AIRBUS  A320-214  2011.0     2016.0      Unknown   Unknown   

             aircraft_type  
0  Fixed Wing Multi-Engine   

Do you want to update mfr_name ? Y/n
n
Do you want to update model ? Y/n
n
Do you want to update year ? Y/n
n
Do you want to update cert_year ? Y/n
n
Do you want to update eng_mfr_name ? Y/n
Y
Input a new eng_mfr_name or Unknown:
CFM INTL
Do you want to update eng_model ? Y/n
Y
Input a new eng_model or Unknown:
CFM56-5B4
Do you want to update aircraft_type ? Y/n
n
  tail_num mfr_name     model    year  cert_year eng_mfr_name  eng_model  \
0   N841VA   AIRBUS  A320-214  2011.0     2016.0     CFM INTL  CFM56-5B4   

             aircraft_type  
0  Fixed Wing Multi-Engine   

Do you want to update this to result? Y/n
Y
Input another tail num? Y/n
Y
Input the tail number:
N271LV
Current info for this tail number:

  t

Do you want to update this to result? Y/n
Y
Input another tail num? Y/n
Y
Input the tail number:
N8707P
Current info for this tail number:

  tail_num mfr_name  model    year  cert_year eng_mfr_name eng_model  \
0   N8707P   BOEING  737-8  2017.0     2017.0      Unknown   Unknown   

             aircraft_type  
0  Fixed Wing Multi-Engine   

Do you want to update mfr_name ? Y/n
n
Do you want to update model ? Y/n
n
Do you want to update year ? Y/n
n
Do you want to update cert_year ? Y/n
n
Do you want to update eng_mfr_name ? Y/n
Y
Input a new eng_mfr_name or Unknown:
CFM INTL
Do you want to update eng_model ? Y/n
Y
Input a new eng_model or Unknown:
LEAP-1B28
Do you want to update aircraft_type ? Y/n
n
  tail_num mfr_name  model    year  cert_year eng_mfr_name  eng_model  \
0   N8707P   BOEING  737-8  2017.0     2017.0     CFM INTL  LEAP-1B28   

             aircraft_type  
0  Fixed Wing Multi-Engine   

Do you want to update this to result? Y/n
Y
Input another tail num? Y/n
Y
Input t

CFM INTL
CFM56-7B24
CFM56-7B27E
LEAP-1B28
  
CFM INTL
CFM56-5B4      for VX320 FR320
CFM56-7B24     for WN737 UA737
CFM56-7B26
CFM56 SERIES   for AS734 WN733 DL319 AA319-1 AA321-1 UA737 UA738 FR319
CFM56-7B27E    for AS739ER DL739ER WN738
CFM56-7B26/3   for DL737 UA739ER
LEAP-1B28      for WN7M8

GE
CF34-10E6       for AAERJ190 B6ERJ190 
CF34 SERIES     for EVCL-600 OOCL-600
CF6-80 SERIES   for DL763-1
GE90-115B       for 77W
GENX-1B70       for 788
GENX-1B74/75/   for 789


IAE
V2500SERIES     for AA319-2 AA320 B6320 DLMD90 NK319 UA320
V2533-A5        for AA321-2
V2527-A5        for NK320

P & W
PW2037          for DL752
PW2040          for UA752
PW4000 SER      for UA744  UA763 UA772 AA333 DL333 DL744
PW4060          for HA763

ROLLS-ROYC
RR772B-60   for HA332 AA332
BR 700 SERIES   for DL717
AE 3007A        for EVE135
AE3007 SER     for EVE145

In [377]:
df_with_carrier.to_csv('./tail_num/tail_num_data_with_carrier.csv', index=False)